# Rapid Implementation of Hardware Neural Networks Powered by Verython


## Project Introduction
For our ECE 5760 final project, we built a Python to Verilog transpiler called ***Verython*** and then used it to implement a convolutional neural network (CNN) on the FPGA to classify handwritten digits. CNNs are among the most popular neural network architectures for image classification, as they have been shown to outperform humans in clinical imaging [(Source)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6586983/), and have a variety of recognition applications! Interestingly, CNNs behave like a black box: they ingest an input and generate an output without revealing their intrinsic logic. This project aimed to demystify CNNs by breaking their layers down into constituent components and implementing them in hardware. Additionally, we wanted to build a library that converts neural networks in Python (the most widely adopted language for machine learning) to Verilog with the hope of making high-performance computing more accessible to everyone.


## TL;DR
Here is a clip of Dr. Adams testing out our project:
![video][hunter drawing inputs](https://www.youtube.com/watch?v=ZbLdQtHRYA0)

Here is a sleep deprived-explanation of our project in the form of a video:
![video][sleep deprived explanation](https://www.youtube.com/watch?v=LqQ_LQ5l_7k)


## High-Level Design


The term "convolutional" is derived from convolution's mathematical operation, which conventionally involves multiplying elements together and then summing them together. Therefore, the output of a convolution is a description of how the shape of one function influences another function. This unique property of our convolution is what our model captures: the spatial dependencies of an image. For our digit classification task, a conventional neural network would be overfitting to the specific location of the digit on the screen. In contrast, the convolutional layer would learn the edges and gradients of the image. 

The architecture of our network is as follows: a series of convolutional layers with max-pooling (twice), followed by a flattened, fully connected layer. Below is the TensorFlow (python library for building neural networks) summary of our model, highlighting all the trained parameters.  

![image info](./images/tf.png)



In the following examples, we describe the high-level operations of each layer.

Our convolutional layer uses a kernel size of $2\times2$.  In this example, we output a $3\times3$ feature map from a $4\times4$ image.  Our $2\times2$ filter slides from left to right across the image, accumulating the element-wise product summation.  We can extrapolate this result to $N\times N$ images with $2\times2$ kernels $\to$ the output feature map will always be $(N-1)\times(N-1)$, given that the kernel is sized $2\times2$.  


![image info](./images/conv.png)


As a finishing touch to the output of our convolutional layer, we pass our values through a special function to “activate” or realize our predictions. Each element of the feature map gets passed through an activation function before moving on to the next layer. We initially chose the sigmoid activation function to keep our values between [0,1]. Extremely large positive values get normalized closer to 1, while large negative values get normalized closer to 0. We soon realized we needed many decimal bits in our fixed-point representation. We ended up resorting to ReLU activation, as the computational cost was cheaper and did not require significant decimal precision to activate values. In this activation function, negative values become zero while positive values retain their original value.

![image info](./images/sigmoid.png)


Immediately following the convolutional layer is our max-pooling layer. Let’s take a 4x4 feature map as an example. Here, we use a pool size of 2x2 with a stride of 2. At each stride, we take the maximum element of the 2x2 window. This results in an output image of size 2x2. When the pool size is equal to the stride size, we effectively reduce the dimensions of the feature map by a factor of 2.  


![image info](./images/mp.png)


Lastly, our fully-connected layer is the flattened output of the feature map. We connect it to our predicted digits of 0-9 and train our model to learn the weights of this layer. We can think of the weights as the strength of the connection. So, how much does the output depend on its connection? We take the dot product of the flattened feature map with each of their respective weights and pass them through another activation function to generate our predictions. We take the index of the element with the maximum probability in our ten-element vector array as our predictions for a given input.

![image info](./images/fullyconnect.png)


## Introduction to Notation
For some array $A$, let $A:[d1,\dots,d_i,\dots,d_n]$ indicate that that $A$ is $n$-dimensional and each dimension $1\leq i\leq n$ has a length of $d_i$. For example,
$$A:[2, 3]=\begin{bmatrix} a_{1, 1} & a_{1,2} & a_{1,3} \\ a_{2,1} & a_{2,2} & a_{2,3} \end{bmatrix}.$$




## Introduction to Verython
Complex structures such as neural networks comprise high-dimensional arrays and a large number of operations. Manually writing our model in Verilog would have been impossible under the three week time constraints--doing so would require tens of thousands of lines of handtyped Verilog. Instead, we wrote a Python library to write the Verilog for us. 

***[Verython](https://github.com/jfw225/verython)*** (portmanteau of Verilog and Python pronounced "verithon") is a Python to Verilog transpiler which generates Verilog modules from objects created in Python. Additionally, it interfaces directly with the free version of Intel's ModelSim to compile the Verilog, simulate the module's waveforms, and export that data back to Python where it compares the simulated data with the expected data computed in Python.


<!-- container: dark -->
#### Syntax 
We start with the notion that lines of code in Verilog (or most languages for that matter) are simply sentences, and we can represent a sentence in Python as a `string`. Thus, if we can generate strings, we can generate Verilog. As we get into the structures in the library, you may notice the use of some types. These types are all defined in **Appendix B: Verython**, and we encourage the reader to reference the appendix in the case of confusion.


##### V_Block
The most fundamental object in ***Verython*** is the `V_Block` object:

In [ ]:
# python
class V_Block(List[str]):
    """
    The type representing a block of verilog code tabbed to the relative level.
    """

    def __init__(self, *lines: List[str]):
        super().__init__([V_Line(line) for line in lines])

A `V_Block` object is essentially a list of `string` objects where each `string` is some line of Verilog code. For instance, the following `V_Block` instance

In [ ]:
# python
width = 4
V_Block(
    f"wire [{width - 1}:0] w;",
    # `*` unpacks each of the strings in the list
    *[f"assign w[{i}] = 1'b1;" for i in range(width)]
)

generates the following verilog code

In [ ]:
# verilog
wire [3:0] w;
assign w[0] = 1'b1;
assign w[1] = 1'b1;
assign w[2] = 1'b1;
assign w[3] = 1'b1;

Obviously, this alone is nothing special, but it provides an easy, readible way to group lines of code and rapidly generate a large amount of Verilog. 


##### Core Blocked Syntax
Since Verilog can be represented by a group of `string` objects, we can represent each of Verilog's core blocked syntax as `V_Block` objects. Here we will show the ***Verython*** version of Verilog Always and If/Else statements, but the rest of the core blocked syntax can be seen in **Appendix B: Verython**.

In [ ]:
# python
def V_Always(
    edge: V_PosEdge or V_NegEdge,
    signal: V_Expression or V_Port or V_Variable
):

    def build(*lines: V_Block) -> V_Block:
        return V_Block(
            f"always @ ({edge()} {signal.name}) begin",
            *[f"\t{line}" for line in lines],
            "end"
        )

    return build
    
def V_If(
    predicate: V_Expression or V_ObjectBase
):

    assert isinstance(predicate, (V_Expression, V_ObjectBase)
                      ), f'"{predicate}" is not a valid predicate.'

    def build(*lines: Iterable[V_Line]) -> V_Block:
        return V_Block(
            f"if ({predicate}) begin",
            *[f"\t{line}" for line in lines],
            "end"
        )

    return build


def V_Else(
    *lines: Iterable[V_Line]
) -> V_Block:

    return V_Block(
        f"else begin",
        *[f"\t{line}" for line in lines],
        "end"
    )

All of the blocked syntax overloads return `V_Block` object, and since a `V_Block` object inherits all of the properties of a Python list, we can use the `*` operator to unpack a `V_Block` into another `V_Block`. For instance, the following two examples are equivalent.

In [ ]:
# python
V_Block(
    "wire clk;",
    "reg [3:0] w;",
    *V_Always(V_Posedge, "clk")(
        *V_If("w == 4'd0")(
            "w <= 4'd1;"
        )
    )
    
) # is equivalent to

V_Block(
    "wire clk;",
    "reg [3:0] w;",
    "".join(*V_Always(V_Posedge, "clk")(
        "".join(V_If("w == 4'd0")(
            "w <= 4'd1;"
        ))
    ))
)

Generated Verilog:

In [ ]:
# verilog
wire clk;
reg [3:0] w;
always @(posedge clk) begin
    if (w == 4'd0) begin
        w <= 4'd1;
    end
end

All of ***Verython*** syntax can be combined in `V_Block` objects to create complex, powerful structures that would be very tedious and cumbersome to manually write in Verilog. 


<!-- container: default -->
#### Objects 
Up until now, you may have noticed that number, wire, and register definitions usage were manually typed in Python `string` objects. However, this disappears with the introduction of ***Verython*** objects.

##### V_Int and V_FixedPoint

In [ ]:
# python
class V_Int(V_Expression):
    """ full implementation not shown """

class V_FixedPoint(V_Expression):
    """ full implementation not shown """

Instances of `V_Int` and `V_FixedPoint` objects keep track of the required bit widths. This is used by ***Verython*** to convert these instances to the proper Verilog format and add an additional layer of error checking before compilation. In addition, Python allows the developer to overload each of the standard library operators for an object. We leveraged this to make the following equivalence possible.

In [ ]:
# python
a = V_Int(1, width=4)
b = V_FixedPoint(1, int_width=1, dec_width=3)

V_Block(
    "wire [3:0] a, b;",
    f"assign a = {a};", 
    f"assign b = {b};"
) # is equivalent to 

V_Block(
    "wire [3:0] a, b;",
    "assign a = 4'b0001;",
    "assign b = 4'b1_000;"
)

Generated Verilog:

In [ ]:
# verilog
wire [3:0] a, b;
assign a = 4'd1;
assign b = 4'b1_000;


##### V_Port, V_Variable, and V_Array

In [ ]:
# python
class V_ObjectBase:
    """ full implementation not shown """

class V_Port(V_ObjectBase):
    """ full implementation not shown """

class V_Variable(V_ObjectBase):
    """ full implementation not shown """

class V_Array(V_ObjectBase, metaclass=V_ArrayMeta):
    """ full implementation not shown """

***Verython*** also has an object representation of Verilog ports, wires, registers, and arrays. Similar to `V_Int` and `V_FixedPoint`, each of the ***Verython*** objects defined above keep track of bit widths and have their default Python operators overloaded. This enables the  ***Verython*** to make the following transcompilation:


In [ ]:
# python
a = V_Int(1, width=4)
b = V_FixedPoint(1, int_width=1, dec_width=3)

pclk = V_Port(module=None,
              port_type=V_Input,
              name="clk")
preset = V_Port(module=None,
                port_type=V_Input,
                name="reset")

pcounter = V_Port(module=None,
              port_type=V_Output,
              dtype=V_Reg,
              width=2,
              name="counter")

wa = V_Variable(module=None,
                dtype=V_Wire,
                width=4,
                name="example_wire")

rb = V_Variable(module=None,
                dtype=V_Reg,
                width=4
                signed=True,
                name="example_reg")

arc = V_Array(module=None,
              dtype=V_RegArray,
              width=4,
              size=2,
              name="example_array")

V_Block(
    "module count_to_four(",
    pclk, preset,
    pcounter,
    ");",
    wa, rb, arc,

    *V_Always(V_PosEdge, pclk)(
        *V_If(preset)(
            pcounter.set(0)

            rb.set(0),

            arc[0].set(a),
            arc[1].set(b),

        ), *V_Else(
            pcounter.set(pcounter + 1),

            rb.set(rb + 1),

            arc[rb].set(wa)
        ),

        *V_If(rb + 1 == 2)(
            rb.set(0)
        )
    ),

    wa.set(rb),
    "endmodule"
) 

Generated Verilog:

In [ ]:
# verilog
module count_to_four(
    input clk, reset,
    output reg [1:0] counter
);
    wire [3:0] example_wire;
    reg signed [3:0] example_reg;
    reg [3:0] example_array [1:0];

    always @(posedge clk) begin
        if (reset) begin
            counter <= 2'd0;
            
            example_reg <= 4'd0;

            example_array[0] <= 4'd1;
            example_array[1] <= 4'b1_000;
        end else begin
            counter <= counter + 2'd1;

            example_reg <= example_reg + 4'd1;

            example_array[example_reg] <= example_wire;
        end

        if (example_reg + 4'd1 == 4'd2) begin
            example_reg <= 4'd0;
        end
    end

    assign example_wire = example_reg;

endmodule


<!-- container: dark -->
#### Modules

If you were at all concerned that you were going to have to manually write out the module header and footer, let me put your mind at ease. ***Verython*** also enables you to write Verilog modules entirely in Python. For the full implementation, look at **Appendix B: Verython**.

[V_Module](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vmodule.py)

In [ ]:
# python
class V_Module:
    """
    The base class for a verilog module.
    """

class V_Iterable(V_Module):
    """
    An object that implements an iterable verilog data structure. 

    Attributes:
        `clk` -- 1-bit input signal
            The port connecting this module to the clock line.

        `reset` -- 1-bit input signal
            The port that enables the caller to reset this module. 

        `write_en` -- 1-bit input signal
            When this signal is HIGH, the data loaded onto `write_data` is 
            stored in memory at address `write_addr`.

        `read_addr` -- (ceil(log2(size)))-bit input signal
            The address from which data is read.

        `write_addr` -- (ceil(log2(size)))-bit input signal
            The address to which data is written.

        `read_data` -- (width)-bit output signal
            The data line into which data is read from memory at address 
            `read_addr`.

        `write_data` -- (width)-bit input signal
            The data line which is written to memory at address `write_addr`.
    """

class V_Target(V_Module):
    """
    The implementation of a module that can be used as the target 
    function in any module. 

    Attributes:
        `clk` -- 1-bit input signal
            The port connecting this module to the clock line.

        `reset` -- 1-bit input signal
            The port that enables the caller to reset this module. 

        `valid` -- 1-bit input signal
            The port that indicates whether or not the input data is valid. The
            caller can manipulate this signal to indicate whether or not the 
            input data is valid.

        `done` -- 1-bit output signal
            The port indicating whether or not the target function has finished 
            computation for a given input. When this flag is `HIGH`, the output 
            data is valid. Thus, the target module should raise this signal 
            when it finishes its task.

        `ready` -- 1-bit output signal
            The port indicating whether or not the target module is ready 
            to begin a task. 


As an example, we will show a ***Verython*** implementation of an M10K block.

In [ ]:
# python
class M10K(V_Iterable):

    def __init__(
        self,
        width: BitWidth,
        size: ArraySize,
        **kwargs,
    ):
        super().__init__(width, size, **kwargs)

        self.memory = V_Array(self, V_RegArray, self.width,
                              self.size, name="memory")

        self.syn_style = '/* synthesis ramstyle = "no_rw_check, M10K" */'

    def generate(self):

        mem_fmt_base, *_ = self.memory.define().split(";")

        return V_Block(
            "// force M10K ram style",
            f'{mem_fmt_base} {self.syn_style} ;'
            "\n",
            *V_Always(V_PosEdge, self.clk)(
                *V_If(self.write_en)(
                    self.memory.set(self.write_addr, self.write_data)
                ),
                self.read_data.set(self.memory.get(self.read_addr))
            )
        )

M10K(width=16, size=784).generate()

Generated Verilog from an instance of `M10K` with `width=16` and `size=784`:

In [ ]:
# verilog
module M10K(
	input    clk,
	input    write_enable,
	input   [9:0] read_addr,
	input   [9:0] write_addr,
	output reg  [15:0] read_data,
	input   [15:0] write_data
);
	// force M10K ram style
	reg  [15:0] memory [783:0]  /* synthesis ramstyle = "no_rw_check, M10K" */;

	always @ (posedge clk) begin
		if (write_enable) begin
			memory[write_addr] <= write_data;
		end
		read_data <= memory[read_addr];
	end
endmodule


<!-- container: default -->
#### States and State Machines
The final piece of the architecture that we'll discuss in this elaborate introduction is ***Verython*** state machines. 


##### V_State

In [ ]:
# python
class V_State(metaclass=_V_State_Meta):
    """
    The object representing a state in a verilog state machine.
    """

    def __init__(self, state_id: V_StateID, width: BitWidth) -> None:
        self.state_id = state_id
        self.width = width

    def generate(self, m: V_Module) -> V_Block:
        """
        This function comprises the state logic: both what is done in the state
        and how the state transitions. The return value should be some verilog
        code and at least one state. The state machine will iterate through
        the returned list of code and replace each `Type[V_State]` with an
        assignment to change the state value. This should be overloaded.

        If no state is found, the next state will be `V_StDone` and the
        module's `done` flag will be raised.

        The parameter `module` is the `V_Module` object in which this state
        will be used.
        """

        return V_Block(
            *V_If(V_Expression(format_int(V_High, 1)))(
                V_State
            )
        )


class V_StateMachine:
    """ 
    Verilog implementation of a finite state machine.
    """

    def __init__(
        self,
        reset_state: Type[V_State],
        *states: Iterable[Type[V_State]]
    ) -> None:

        # determine bit width needed for a state variable
        # (+ 1 is for `V_StDone`)
        self.width = BitWidth(ceil(log2(len(states) + 1)))

        # set the reset state
        self._reset_state = reset_state(
            state_id=V_StateID(0), width=self.width)

        # maps class types to initialized objects
        self._state_map: Dict[NetName, V_State] = {
            str(state): state(state_id=V_StateID(i), width=self.width)
            for i, state in enumerate(states)
        }

    def generate(
        self,
        module: V_Module,
        clk: V_Clock,
        reset: V_Reset,
        done: V_Done,
        edge: Optional[V_PosEdge or V_NegEdge] = V_PosEdge
    ) -> V_Block:
        """
        This function generates a verilog state machine from `self._state_map` 
        that operates on the edge `edge of clock `clk`.

        Each state has a `generate` function that should return an iterable 
        containing lines of verilog and at least `V_State` object. If no 
        `V_State` object is found, an error will be thrown.  

        This function will create a state variable `state` in the verilog 
        module `module`, and will assign each of the states a value. We will 
        initialize `state` to `self._start_state` when `reset` is HIGH. 

        Let `V_i` denote the value assigned to the `i`-th state `S_i`. Then this 
        function will search through the iterables returned from each of the 
        `V_State.generate` calls and replace each `S_i` with `state <= V_i`.

        The state machine will continue to run until `V_StDone` is 
        reached--at which point, the state machine will raise `done` and idle 
        until `reset` is set.
        """


States in any sort of finite state machine usually have two roles: do something and transition to a state. 

The "do something" component is accomplished by providing each state with the module `m` that implements the state machine. Doing so gives the state access to all of the modules ports, variables, and instances. 

The transition part is a little tricky. Suppose when you make your state machine, you want some state $a$ to transition to some other state $b$, and once you get to $b$, you want to go back to $a$. This is implemented in the following pseudocode:

In [ ]:
# python
class StateA:
    def __init__(self, state_b):
        self.state_b = state_b

    def transition(self):
        do_something()

        go_to_state(self.state_b)

class StateB:
    def __init__(self, state_a):
        self.state_a = state_a

    def transition(self):
        do_something()

        go_to_state(self.state_a)

state_a = StateA(state_b=None)
state_b = StateB(state_a=state_a)

Well $a$ and $b$ are really referring to instances of some state object. Suppose you create $a$ first. Well $a$ needs to have a reference to $b$, but $b$ hasn't been instansiated yet. Same logic applies if you create $b$ first. We solve this issue by instead referencing states by their static type rather than their instance. And then at transpile-time, the `V_StateMachine` object creates some `state` register, assigns each referenced state type some number `n`, and replaces the static reference with `state <= n` inside of an always block. Our example above becomes:

In [ ]:
# python
class OnReset(V_State):
    def generate(self, m: V_Module) -> V_Block(

        return V_Block(
            do_something(),

            StateA
        )
    )

class StateA(V_State):
    def generate(self, m: V_Module) -> V_Block(

        return V_Block(
            do_something(),

            StateB
        )
    )

class StateB(V_State):
    def generate(self, m: V_Module) -> V_Block(

        return V_Block(
            do_something(),

            StateA
        )
    )

# notice the state parameters are not instances, they are static types
V_StateMachine(OnReset, StateA, StateB).generate()

Generated Verilog:

In [ ]:
# verilog
reg [1:0] state;

always @(posedge clk) begin
    if (reset) begin
        // do something

        // go to StateA
        state <= 2'd0;
    end
    else begin 
        case (state)
            // StateA
            2'd0: begin
                    // do something

                    // go to StateB
                    state <= 2'd1;
                end
            
            // StateB
            2'd1: begin
                    // do something 

                    // go to StateA
                    state <= 2'd0;
                end
        endcase
    end
end

Combining the tools provided in the ***Verython*** library enables the rapid creation of robust and expansive Verilog code. In fact, let's talk about how we used it to do just that.

## Program/Hardware Design


### Model Implementation
In this section, we will dive deep into the math behind each of the layers in the model and how we rederived the transformation functions of each layer to fit on the FPGA. 

As a reminder, our model ingests some image of a hand-drawn digit and outputs a number between 0 and 9 which corresponds to the prediction of the model. The image is a 28 by 28 2D array of values between 0 and 255, or rather, some input image $img$ can be represented as
$$img:[n=28,n]=\begin{bmatrix} 
  p_{1,1} & \dots & p_{1,j} & \dots & p_{1,28} \\ 
  \vdots & \ddots & & & \vdots \\
  p_{i, 1} & \dots & p_{i, j} & \dots & p_{i, 28} \\
  \vdots & & & \ddots & \vdots \\
  p_{28, 1} & \dots & p_{28, j} & \dots & p_{28, 28}
\end{bmatrix}$$
such that $0\leq p_{i,j}\leq255$. 

Moreover, we can think of our model $M$ as a function that maps some input $img$ to some output prediction $pred$, or rather, the inference of our model can be represented by 
$$M(img)=pred$$
for some output prediction $0\leq pred\leq9$. 

Furtheremore, we can also express each of the layers in our model as a a function. Let $L=\{L_1,\dots,L_i,\dots,L_4\}$ be the layers in our model. Then we can represent each layer $L_i$ as  
$$L_i(V_i^{in})=V_i^{out},$$
where $V_{in}$ and $V_{out}$ are matrices whose shapes are defined by $L_i$. 

Without loss of generality, each layer $L_i$ has associated weights $W^i$ and biases $B^i$. These parameters are applied to $V_i^{in}$ to produce $V_i^{out}$ using set of operations that varies by layer.

With this new notation, we can redfine $M$ in terms of its layers:
$$\begin{align*}
  M(img) & =L_4(L_3(L_2(L_1(img)))) \\
  & =pred.
\end{align*}$$
Thus, a model's prediction is simply the output of its cascaded layer functions.

Let's now take a look at each of the layers and how we can rederive their functions to be built on the FPGA.


<!-- container: dark -->
#### Layer 1: Conv2D
The 2D convolutional layer has weights $W^1$ and biases $B^1$ that are defined by 
$$\begin{align*}
    & W^1:[x=2,y=2,z=8]=
    \begin{bmatrix}
        \begin{bmatrix}
            \begin{bmatrix} o_1 & \dots & o_k & \dots & o_z \end{bmatrix}_{1,1} \\
            \vdots \\
            \begin{bmatrix} p_1 & \dots & p_k & \dots & p_z \end{bmatrix}_{1,y}
        \end{bmatrix}_1 \\
        \vdots \\
        \begin{bmatrix}
            \begin{bmatrix} q_1 & \dots & q_k & \dots & q_z \end{bmatrix}_{x, 1} \\
            \vdots \\
            \begin{bmatrix} r_1 & \dots & r_k & \dots & r_z \end{bmatrix}_{x, y}
        \end{bmatrix}_x
    \end{bmatrix}, \\
    & B^1:[z=8]=\begin{bmatrix} b_1 & \dots & b_z \end{bmatrix}
\end{align*}$$ 
for $k=1,\dots,z$. And since this is the first layer, the input data is the image, or rather 
$$V^{in}_1:[n=28,n]=img,$$
and the output data is defined by $V^{out}_1:[n-1,n-1,z]$.



##### Mathematical Description
So, how does the convolutional layer produce its output? We start by taking two counters $0\leq i,j\leq n-1$ and assigning an array `sub` to a 2D subset of $V_1^{in}$. Specifically, we take the $j$ through $j+y$ elements in the $i$ through $i+x$ rows of $V_1^{in}$. We can rigidly define `sub` as 
$$sub:[x, y]=V^{in}_{1_{i:i+x,~j:j+y}}=
\begin{bmatrix}
    \begin{bmatrix} 
        f_1 & \dots & f_y
    \end{bmatrix}_1 \\
    \vdots \\
    \begin{bmatrix}
        g_1 & \dots & g_y
    \end{bmatrix}_x
\end{bmatrix}.
$$

We then take the dot product of `sub` and $W^1$ with respect to the second and third dimensions which yields 
$$
conv_{i,j}:[z]=
\begin{bmatrix}
    f_1\cdot o_1+\dots+f_y\cdot p_1+g_1\cdot q_1+\dots+g_y\cdot r_1 \\
    \vdots \\
    f_1\cdot o_k+\dots+f_y\cdot p_k+g_1\cdot q_k+\dots+g_y\cdot r_k \\
    \vdots \\
    f_1\cdot o_z+\dots f_y\cdot p_z+g_1\cdot q_z+\dots+g_y\cdot r_z
\end{bmatrix}
$$
for $k=1,\dots,z$.

Lastly, we add the $z$ biases in $B^1$ to the `z` elements in $conv_{i,j}$ and store the resulting vector in the $i$-th row and $j$-th column of the output array:
$$
V_{1_{i,j}}^{out}=
\begin{bmatrix} 
    conv_{i,j,1}+b_1 \\
    \vdots \\
    conv_{i,j,k}+b_k \\
    \vdots \\
    conv_{i,j,z}+b_z
\end{bmatrix}
$$
for $k=1,\dots,z$.


##### Programatic Description
Before we think about the functionality in terms of hardware, we introduce a code block of Python mixed with pseudocode to put the math in context (which I promise is far more readable than the Verilog implementation):

In [ ]:
# python
import numpy as np

def Conv2D(
    V1in: np.ndarray[28, 28], 
    W1: np.ndarray[2, 2, 8], 
    B1: np.ndarray[8]
):  
    n, n = V1in.shape
    x, y, z = W1.shape

    # init the output array
    V1out = np.ndarray[n - 1,  n - 1, z]  # (27, 27, 8)

    for i in range(n - 1):  # rows if image
        for j in range(n - 1):  # columns of image
            # array of shape `(x, y)`
            sub: np.ndarray[x, y] = V1in[i:i + x, j:j + y] 

            # vector of length `z`
            convij: np.ndarray[z] = np.sum(sub * W1, axis=(0, 1))

            V1out[i, j] = convij + B1

    return V1out

##### Verython Implementation

In [ ]:
# python
class Conv2D(Layer):
    """
    img of size (n, n)
    kernel of size (x, y, z)

    We have z=8 filters, each of which are x=2 by y=2
    Let FIL_1 denote the first filter(i, j) using the notation above, then:
    kernel = [ [f1      [h1
                f2       h2
                g1 ,     j1 ...
                g2]      j2]       ]

           = [FIL_1.T, FIL_2.T, ... , FIL_8.T]  --> this is hard to think about
                                            since these filters are transposed

    at each iteration i=1,..., n - 1 and j=1,..., n - 1:
    sub(i, j) of size (x, y) is subset of the image
    sub(i, j) = [[f1, ..., fx],
                 [g1, ..., gy]]

    kernel = [[ F1 = [o1, ..., oz],
                ...
                Fx = [p1, ..., pz]],
              [ G1 = [q1, ..., qz],
                ...
                Gy = [r1, ..., rz]]

    prod(i, j) = sub(i, j) * kernel
               = [[ f1 * F1,
                    ...
                    fx * Fx],
                  [ g1 * G1,
                    gy * Gy]]
    sum(i, j) = prod.sum(axis=(0, 1)) =
        [
            f1 * o1 + ... + fx * p1 + g1 * q1 + ... + gy * r1,
            ...
            f1 * oz + ... + fx * pz + g1 * qz + ... + gy * rz
        ]

    output is array of size (n - 1, n - 1, z) where
    output[i, j] = sum(i, j)
    ------------
    This very abstract representation makes hard to develop the intuition,
    let's ignore this for now
    ------------
    mat is a 2x2 slice of the image --> img[i:i+2, j:j+2]
    Let's say:
    mat = np.array([[[0.1],
                     [0.5]],
                     [[1.],
                     [1.]]])
    kernel = np.ones( (2,2,8) )

    prod(i, j) = mat * kernel
               = np.array([[[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                            [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]],
                            [[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
                            [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ]]])
    output(i, j) = np.array([2.6, 2.6, 2.6, 2.6, 2.6, 2.6, 2.6, 2.6])
                                             --> we sum the columns
    """

    def __init__(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray,
        input_mem: M10K,
        output_mem: M10K,
        input_shape: Optional[InputShape],
        output_shape: Optional[OutputShape]
    ):

        # create the signed mult and sigmoid modules
        self.signed_mult = SignedMult(int_width, dec_width)

        super().__init__(
            int_width=int_width,
            dec_width=dec_width,
            weights_np=weights_np,
            biases_np=biases_np,
            input_mem=input_mem,
            output_mem=output_mem,
            input_shape=input_shape,
            output_shape=output_shape,
            objects=[self.signed_mult]
        )

        # make sure input data is signed
        self.inp_data.signed = True

        # make output data a wire
        self.out_data.dtype = V_Wire

        x, y, z = weights_np.shape

        assert x == y, weights_np

        # the constant used to scale the inputs (>> 8 -> / 256)
        self.input_scaler = 8

        # store the target size (** MIGHT BE SOURCE OF ERROR)
        self.target_size = int(np.sqrt(self.input_mem.size)) - 1
        print(f"{nameof(self)} Target Size: {self.target_size}")

        # the number of rows in the flattened weights
        self.w_rows = x * x

        # create registers used as iteration indices
        # ti=0, ..., target_size - 1
        self.ti = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="ti")

        # tj=0, ..., target_size - 1
        self.tj = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="tj")

        # ii=0, ..., x - 1
        self.ii = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="ii")

        # jj=0, ..., x - 1
        self.jj = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="jj")

        # r=0, ..., x * x - 1
        self.r = self.var(dtype=V_Reg,
                          width=self.input_mem.addr_width,
                          name="r")

        # wc=0, ..., z - 1
        self.wc = self.var(dtype=V_Reg,
                           width=self.w_addr.width,
                           name="wc")

        # create a register array for the sub image
        self.sub = self.var(
            dtype=V_RegArray,
            width=self.width,
            size=x * x,
            signed=True,
            name="sub_img"
        )

        # create a wire to to hold the output of the convolution
        self.conv_out = self.var(
            dtype=V_Wire,
            width=self.width,
            size=self.output_mem.size,
            signed=True,
            name="conv_out"
        )

        # create `self.w_rows` multiplier outputs
        self.mult_outs = [self.var(dtype=V_Wire,
                                   width=self.signed_mult.width,
                                   signed=True,
                                   name=f"mult_out_{wr}")
                          for wr in range(self.w_rows)]

    @staticmethod
    def forward(
        x: np.ndarray,
        weights_np: Weights,
        biases_np: Biases
    ):

        Z1 = convolve_flat(x, weights_np)

        return Z1 + biases_np

    def convert_model_params(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray
    ) -> Tuple[V_Array, V_Array]:

        assert biases_np.ndim == 1, biases_np

        w_flat = weights_np.flatten()

        weights = V_Array(
            module=self,
            dtype=V_ParameterArray,
            width=int_width + dec_width,
            size=len(w_flat),
            signed=True,
            data=[V_FixedPoint(v, int_width, dec_width) for v in w_flat],
            name=f"conv2d_weights"
        )

        biases = V_Array(
            module=self,
            dtype=V_ParameterArray,
            width=int_width + dec_width,
            size=len(biases_np),
            signed=True,
            data=[V_FixedPoint(v, int_width, dec_width)
                  for v in biases_np],
            name=f"conv2d_biases"
        )

        return weights, biases

    def generate(self) -> V_Block:
        x, y, z = self.weights_np.shape

        vsm = V_StateMachine(_StReset, _StWaitValid,
                             _StResetSub, _StSetInpAddr, _StSubBuffer,
                             _StSetSub, _StComputeConv, _StIncWeightIndices,
                             _StIncTargetIndices)

        # instantiate x * x signed mult modules
        mult = self.signed_mult
        self.signed_mult_inss = [self.signed_mult.instantiate(
            self,
            (V_Empty(), mult.clk),
            (V_Empty(), mult.reset),
            (V_Empty(), mult.valid),
            (V_Empty(), mult.done),
            (V_Empty(), mult.ready),
            (self.sub[wr], mult.input_ports[0]),
            (self.weights[self.wc + wr * z], mult.input_ports[1]),
            (self.mult_outs[wr], mult.output_ports[0])
        ) for wr in range(self.w_rows)]

        return super().generate(vsm, V_Block(
            "// instantiate the multipliers",
            *[line for mult_ins in self.signed_mult_inss for line in mult_ins],

            "\n// assign the output of the convolution",
            self.out_data.set(
                V_Sum(*self.mult_outs, self.biases[self.wc]))
            # self.conv_out.set(
            #     V_Sum(*self.mult_outs, self.biases[self.out_addr]))
        ))


"""
Conv2D State Machine:
img of size (n, n)
kernel of size (x, y, z)


first load each input into a local register array
reasoning:
    at each iteration, we need to get 4 new inputs from input memory. this
    would take at least 5 cycles to set input addr and get new data

    getting all the inputs into a reg array would allow us to get new values
    in same cycle

call this reg array `inp_reg`
for iteration i=1,..., n - 1 and j=1,..., n - 1:
filter(i, j) is of size (x, y)


set the sub by 
for ti in range(target_size):
    for tj in range(target_size):
        sub[0:x * x - 1] = inp_reg[ti:ti + x, ti:ti + x]


go straight to computing this the prod.sum:
sum(i, j) = prod.sum(axis=(0, 1)) =
        [
            f1 * o1 + ... + fx * p1 + g1 * q1 + ... + gy * r1,
            ...
            f1 * oz + ... + fx * pz + g1 * qz + ... + gy * rz
        ]
        =
        [
            sub[0] * W[0 + 0 * z] + ... + sub[wr] * W[0 + wr * z],
            sub[0] * W[1 + 0 * z] + ... + sub[wr] * W[1 + wr * z],
            ...
            sub[0] * W[wc + 0 * z] + ... + sub[wr] * W[wc + wr * z]
        ]
where wr=0, ..., x * x - 1 and is the row index and 
      wc=0, ...., z - 1
"""


class _StReset(V_State):
    """
    init input read addr
    init output write addr
    lower output write enable

    init iteration variables

    raise sig reset
    lower sig valid

    go to StWaitValid
    """

    def generate(self, m: Conv2D) -> V_Block:

        return V_Block(
            m.inp_addr.set(m.input_mem.base_addr),
            m.out_addr.set(m.output_mem.base_addr),
            m.out_we.set(V_Low),

            m.ti.set(V_Low),
            m.tj.set(V_Low),
            m.wc.set(V_Low),

            _StWaitValid
        )


class _StWaitValid(V_State):
    """
    if (valid)
        go to StInitInputReg
    """

    def generate(self, m: Conv2D) -> V_Block:

        return V_Block(
            *V_If(m.valid)(

                _StResetSub
            )
        )


class _StResetSub(V_State):
    """
    reset the counters used for sub
    """

    def generate(self, m: Conv2D) -> V_Block:
        return V_Block(
            m.ii.set(V_Low),
            m.jj.set(V_Low),
            m.r.set(V_Low),

            _StSetInpAddr
        )


class _StSetInpAddr(V_State):
    """
    set the input address

    """

    def generate(self, m: Conv2D) -> V_Block:
        n = m.target_size + 1

        return V_Block(
            m.inp_addr.set((V_Par(m.ti + m.ii) * n) + m.tj + m.jj),

            _StSubBuffer
        )


class _StSubBuffer(V_State):
    """
    delay 1 cycle
    """

    def generate(self, m: Conv2D) -> V_Block:
        return V_Block(

            _StSetSub
        )


class _StSetSub(V_State):
    """
    set the sub by 
    for ti in range(target_size):
        for tj in range(target_size):
            sub[0:x * x - 1] = inp_reg[ti:ti + x, ti:ti + x]

            set wc to zero

            go to StComputeConv
    """

    def generate(self, m: Conv2D) -> V_Block:
        # should be the first dim of the input shape
        x, *_ = m.weights_np.shape

        # indices = [(ii * n, jj) for ii in range(k) for jj in range(k)]

        max_ii = x - 1
        max_jj = x - 1
        max_r = x * x - 1

        return V_Block(
            m.sub[m.r].set(m.inp_data),

            *V_If(m.r == max_r)(
                m.wc.set(V_Low),
                _StComputeConv
            ), *V_Else(
                m.r.set(m.r + 1),

                # dont need to handle the case of max_ii because m.r == max_r
                *V_If(m.jj == max_jj)(
                    m.ii.set(m.ii + 1),
                    m.jj.set(V_Low)
                ), *V_Else(
                    m.jj.set(m.jj + 1)
                ),

                _StSetInpAddr
            )

        )


class _StComputeConv(V_State):
    """
    raise output write enable

    go to StComputeSig
    """

    def generate(self, m: Conv2D) -> V_Block:

        return V_Block(
            m.out_we.set(V_High),

            _StIncWeightIndices
        )


class _StIncWeightIndices(V_State):
    """
    clear the output write enable

    if (wc == w_rows - 1)
        set wc to 0

        if (output addr = max output addr - 1)
            go to StDone
        else 
            inc the output addr 

            go to StIncTargetIndices
    else
        inc wc
        inc the output addr

        go to StComputeConv
    """

    def generate(self, m: Conv2D) -> V_Block:
        max_out_addr = m.output_mem.size - 1
        # max_rows = m.w_rows - 1
        x, y, z = m.weights_np.shape
        max_cols = z - 1

        return V_Block(
            m.out_we.set(V_Low),

            *V_If(m.wc == max_cols)(
                m.wc.set(V_Low),

                *V_If(m.out_addr == max_out_addr)(
                    V_StDone
                ), *V_Else(
                    m.out_addr.set(m.out_addr + 1),

                    _StIncTargetIndices
                )
            ), *V_Else(
                m.wc.set(m.wc + 1),
                m.out_addr.set(m.out_addr + 1),

                _StComputeConv
            )
        )


class _StIncTargetIndices(V_State):
    """

    if (m.tj == m.target_size - 1)
        set m.tj to 0

        if (m.ti == m.target_size - 1)
            go to StDone
        else
            inc m.ti
            go to StSetSub
    else
        inc m.tj

        go to StSetSub
    """

    def generate(self, m: Conv2D) -> V_Block:
        max_iter = m.target_size - 1

        return V_Block(
            *V_If(m.tj == max_iter)(
                m.tj.set(V_Low),

                *V_If(m.ti == max_iter)(
                    V_StDone
                ), *V_Else(
                    m.ti.set(m.ti + 1),
                    _StResetSub
                )
            ), *V_Else(
                m.tj.set(m.tj + 1),

                _StResetSub
            )
        )

Generated Verilog: [Layer1](https://github.com/jfw225/mnist-cnn-fpga/blob/main/deployment/DE1-SoC_Computer_15_640_current/verilog/layer1.sv)


<!-- container: default -->
#### Layer 2: Maxpool
This maxpool layer has no weights or biases. Additionally, its input and output are defined by 
$$\begin{align*}
    & V_2^{in}: [x=27, y=27, n=8]=V_1^{out}, \\
    & V_2^{out}:[\lceil x/2\rceil=13, \lceil y/2\rceil=13, n=8].
\end{align*}$$
where $\lceil r\rceil$ is $r$ rounded up to the nearest integer for some $r\in\mathbb{R}$.
We also use $s=2$ for the stride hyperparameter.

##### Mathematical Description
Like with the last layer, we start by taking two counters $0\leq i\leq x,0\leq j\leq y$ and assigning an array `window` to a 2D subset of $V_2^{in}$. Specifically, we take the $j$ through $j+s$ elements in the $i$ through $i+s$ rows of $V_2^{in}$. We can rigidly define `window` as 
$$window_{i,j}:[s, s, n]=V_{2_{i:i+s,j:j+s}}^{in}=
\begin{bmatrix}
    \begin{bmatrix}
        \begin{bmatrix}
            o_1 & \dots & o_k & \dots & o_n
        \end{bmatrix}_{1,1} \\
        \vdots \\
        \begin{bmatrix}
            p_1 & \dots & p_k & \dots & p_n
        \end{bmatrix}_{1,s}
    \end{bmatrix}_1 \\
    \vdots \\
    \begin{bmatrix}
        \begin{bmatrix}
            q_1 & \dots & q_k & \dots & q_n 
        \end{bmatrix}_{s,1} \\
        \vdots \\
        \begin{bmatrix}
            r_1 & \dots & r_k & \dots & r_n
        \end{bmatrix}_{s,s}
    \end{bmatrix}_s
\end{bmatrix}.
$$

We then take the column-wise maximum of `window` (i.e. the max of the elements aligned vertically in the array above) which yields
$$
max_{i,j}:[n]=
\begin{bmatrix}
    \max\left\{o_1,\dots,p_1,q_1,\dots,r_1\right\} \\
    \vdots \\
    \max\left\{o_k,\dots,p_k,q_k,\dots,r_k\right\} \\
    \vdots \\
    \max\left\{o_n,\dots,p_n,q_n,\dots,r_n\right\}
\end{bmatrix}$$
for $k=1,\dots,n$.

Lastly, we take the length-$n$ vector $max_{i,j}$ and store it in the $\lceil i/s\rceil$-th row and the $\lceil j/s\rceil$-th column of the output array:
$$V_{1_{\lceil i/s\rceil,\lceil j/s\rceil}}^{out}=max_{i,j}.$$
After each iteration, we increment $j$ by $s$ and repeat this until $j+s>y$. At this point, we increment $i$ by $s$ and continue to iterate until $i+s>x$. When this is true, the pooling has completed. This stop condition will be come more clear in the *Programatic Description*.


##### Programatic Description
Like before, we introduce a code block of Python mixed with pseudocode to put the math in context:

In [ ]:
# python
import numpy as np

def Maxpool(V2in: np.ndarray[27, 27, 8], s=2):
    x, y, n = image.shape

    V2out = np.ndarray[x // 2, y // 2, n]

    # slide the window over every part of the image using stride s
    # take the maximum value at each step
    i = 0

    # slide the max pooling window vertically across the image
    while i + s <= x:
        j = 0

        # slide the max pooling window horizontally across the image
        while j + s <= y:
            # choose the max value in the window at each step
            maxij = np.max(V2in[i:i + s, j:j + s], axis=(0, 1))

            # store it to output matrix
            V2out[i // s, j // s] = maxij
            
            # increment `j`
            j += s

        # increment `i`
        i += s

    return V2out

##### Verython Implementation

In [ ]:
# python
class Maxpool(Layer):
    """
    Let the input to this layer be

    mp_input of size (y, x, n)
    window is of size (f * f * n)

    window = [[
        A1 = [o1, ..., on],
        ...,
        Af = [p1, ..., pn]
    ], [
        B1 = [q1, ..., qn],
        ...,
        Bf = [r1, ..., rn]
    ]]

    the output at iteration (i, j) is the columnwise max of
    [A1, ..., Af, B1, ..., Bf], or rather,
    max(i, j) of size (n) = [
        max(o1, ..., p1, q1, ..., r1),
        ...,
        max(on, ..., pn, qn, ..., rn)
    ] = [
        max(
            window[0 + 0 * z * z]
            + ... + window[0 + wr * z * z] + ... +
            window[0 + (z * z - 1) * z * z]
        ), ...,
        max(
            window[wc + 0 * z * z]
            + ... + window[wc + wr * z * z] + ... +
            window[wc + (z * z - 1) * z * z]
        ), ...,
        max(
            window[(n - 1) + 0 * z * z]
            + ... + window[(n - 1) + wr * z * z] + ... +
            window[(n - 1) + (z * z - 1) * z * z]
        )
        max(window[wc + wr * z * z])
    ]

    where wc=0, ..., n - 1 is the column sweep and
    wr=0, ..., z * z - 1 is the row sweep.
    """

    def __init__(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray,
        input_mem: M10K,
        output_mem: M10K,
        input_shape: Optional[InputShape],
        output_shape: Optional[OutputShape]
    ):

        # create the max. mult, exponential, and div modules
        self.signed_max = SignedMax(int_width, dec_width)

        super().__init__(
            int_width=int_width,
            dec_width=dec_width,
            weights_np=weights_np,
            biases_np=biases_np,
            input_mem=input_mem,
            output_mem=output_mem,
            input_shape=input_shape,
            output_shape=output_shape,
            objects=[self.signed_max]
        )

        # configure the output data as a wire
        self.out_data.dtype = V_Reg

        # store f and stride values
        self._f = 2
        self._s = 2

        # we assume the f is even for now
        assert self._f % 2 == 0, self._f

        # create a register array for the sliding window
        self.window = self.var(dtype=V_RegArray,
                               width=self.width,
                               size=self.f * self.f * self.n,
                               signed=True,
                               name="window")

        # create iteration variables as in `maxpool_flat`
        self.curr_y = self.var(dtype=V_Reg,
                               width=self.input_mem.addr_width,
                               name="curr_y")
        self.out_y = self.var(dtype=V_Reg,
                              width=self.output_mem.addr_width,
                              name="out_y")

        self.curr_x = self.var(dtype=V_Reg,
                               width=self.input_mem.addr_width,
                               name="curr_x")
        self.out_x = self.var(dtype=V_Reg,
                              width=self.output_mem.addr_width,
                              name="out_x")

        # jj=0, ..., f - 1
        self.jj = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="jj")

        # ii=0, ..., f - 1
        self.ii = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="ii")

        # kk=0, ..., n - 1
        self.kk = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="kk")

        # ww=0, ..., f * f * n - 1
        self.ww = self.var(dtype=V_Reg,
                           width=ceil(log2(self.window.size)),
                           name="ww")

        # wc=0, ..., n - 1
        self.wc = self.var(dtype=V_Reg,
                           width=self.output_mem.addr_width,
                           name="wc")

        # wr=0, ..., f * f - 1
        self.wr = self.var(dtype=V_Reg,
                           width=ceil(log2(self.window.size)),
                           name="wr")

        # holds the output of the signed max module
        self.max_out = self.var(dtype=V_Wire,
                                width=self.width,
                                signed=True,
                                name="max_out")

    @property
    def f(self):
        """
        The `f` parameter in `maxpool_flat`.
        TODO: KEN COME BACK AND EXPLAIN THIS
        """

        return self._f

    @property
    def s(self):
        """
        The stride parameter in `maxpool_flat`.
        """

        return self._s

    @property
    def y(self):
        """
        Input array is of size `(y, x, n)`.
        """

        y, x, n = self.input_shape

        return y

    @property
    def x(self):
        """
        Input array is of size `(y, x, n)`.
        """

        y, x, n = self.input_shape

        return x

    @property
    def n(self):
        """
        Input array is of size `(y, x, n)`.
        """

        y, x, n = self.input_shape

        return n

    @staticmethod
    def forward(
        x: np.ndarray,
        weights_np: Weights,
        biases_np: Biases
    ):

        return maxpool_flat(x)

    def convert_model_params(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray
    ) -> Tuple[V_Array, V_Array]:

        return None, None

    def generate(self) -> V_Block:

        vsm = V_StateMachine(_StReset, _StWaitValid,
                             _StResetWindow, _StSetInpAddr,
                             _StWindowBuffer, _StSetWindow,
                             _StMaxReset, _StGetMaxOverWindow,
                             _StFindMax, _StMaxBuffer,
                             _StIncMaxpool, _StWriteData)

        sm = self.signed_max

        return super().generate(vsm, V_Block(
            "// instantiate the signed maximum module",
            *self.signed_max.instantiate(
                self,
                (V_Empty(), sm.clk),
                (V_Empty(), sm.reset),
                (V_Empty(), sm.valid),
                (V_Empty(), sm.done),
                (V_Empty(), sm.ready),
                (self.window[self.wc], sm.input_ports[0]),
                (self.window[self.wc + self.wr * self.n], sm.input_ports[1]),
                (self.max_out, sm.output_ports[0])
            )
        ))


class _StReset(V_State):
    """
    init input read addr
    init output write addr
    lower output write enable

    init iteration variables

    go to _StWaitValid
    """

    def generate(self, m: Maxpool) -> V_Block:

        return V_Block(
            m.inp_addr.set(m.input_mem.base_addr),
            m.out_addr.set(m.output_mem.base_addr),
            m.out_we.set(V_Low),

            m.curr_y.set(V_Low),
            m.out_y.set(V_Low),
            m.curr_x.set(V_Low),
            m.out_x.set(V_Low),


            _StWaitValid
        )


class _StWaitValid(V_State):
    """
    if (valid)
        go to StInitInputReg
    """

    def generate(self, m: Maxpool) -> V_Block:

        return V_Block(
            *V_If(m.valid)(

                _StResetWindow
            )
        )


class _StResetWindow(V_State):
    """
    """

    def generate(self, m: Maxpool) -> V_Block:
        return V_Block(
            m.jj.set(V_Low),
            m.ii.set(V_Low),
            m.kk.set(V_Low),
            m.ww.set(V_Low),

            _StSetInpAddr
        )


class _StSetInpAddr(V_State):
    """
    """

    def generate(self, m: Maxpool) -> V_Block:

        return V_Block(
            m.inp_addr.set(
                (V_Par(m.curr_y + m.jj) * m.x * m.n) +
                (V_Par(m.curr_x + m.ii) * m.n) +
                m.kk),

            _StWindowBuffer
        )


class _StWindowBuffer(V_State):
    """
    """

    def generate(self, m: Maxpool) -> V_Block:

        return V_Block(

            _StSetWindow
        )


class _StSetWindow(V_State):
    """
    set window = inp_reg[curr_y:curr_y + f, curr_x:curr_x + f]

    inp_reg of size(y, x, n)
    inp_reg[j, i, k] = inp_reg[(j * x * n) + (i * n) + k]

    need to sweep
    j=curr_y, ..., curr_y + f
    i=curr_x, ..., curr_x + f
    k=0, ..., n

    go to _StFindMax
    """

    def generate(self, m: Maxpool) -> V_Block:
        max_jj = m.f - 1
        max_ii = m.f - 1
        max_kk = m.n - 1
        max_ww = m.f * m.f * m.n - 1

        return V_Block(
            m.window[m.ww].set(m.inp_data),

            *V_If(m.ww == max_ww)(

                _StMaxReset
            ), *V_Else(
                m.ww.set(m.ww + 1),

                *V_If(m.kk == max_kk)(
                    m.kk.set(V_Low),

                    # dont need to set jj low because that is case
                    # where m.ww == max_ww
                    *V_If(m.ii == max_ii)(
                        m.ii.set(V_Low),
                        m.jj.set(m.jj + 1)
                    ), *V_Else(
                        m.ii.set(m.ii + 1)
                    )
                ), *V_Else(
                    m.kk.set(m.kk + 1)
                ),

                _StSetInpAddr
            )
        )


class _StMaxReset(V_State):
    """
    """

    def generate(self, m: Maxpool) -> V_Block:

        return V_Block(
            m.wc.set(V_Low),
            m.wr.set(V_High),

            _StGetMaxOverWindow
        )


class _StGetMaxOverWindow(V_State):
    """
    """

    def generate(self, m: Maxpool) -> V_Block:
        max_wr = m.f * m.f - 1

        return V_Block(
            m.window[m.wc].set(m.max_out),

            *V_If(m.wr == max_wr)(

                _StFindMax
            ), *V_Else(
                m.wr.set(m.wr + 1),

                _StGetMaxOverWindow
            )
        )


class _StFindMax(V_State):
    """
    assign each of the column-wise maximums to the mp_out register

    mp_out of size (y // 2, x // 2, n)

    mp_out[(out_y * x // 2 * n) + (out_x * n) + wc] = max_outputs[wc]
    for window column index wc=0,..., n - 1.
    """

    def generate(self, m: Maxpool) -> V_Block:

        return V_Block(
            m.out_we.set(V_Low),
            m.out_addr.set(
                (m.out_y * (m.x // 2) * m.n) +
                (m.out_x * m.n) +
                m.wc
            ),
            # m.out_data.set(m.max_out_arr[m.wc]),
            m.out_data.set(m.max_out),

            _StMaxBuffer
        )


class _StMaxBuffer(V_State):
    """
    """

    def generate(self, m: Maxpool) -> V_Block:
        max_wc = m.n - 1

        return V_Block(
            m.out_we.set(V_High),

            *V_If(m.wc == max_wc)(
                _StIncMaxpool
            ), *V_Else(
                m.wc.set(m.wc + 1),
                m.wr.set(1),

                _StGetMaxOverWindow
            )

        )


class _StIncMaxpool(V_State):
    """
    set each of the previously set mp_outs to to themselves

    if (curr_x <= x - s - f)
        set curr_x to 0
        set out_x to 0

        if (curr_y <= y - s - f)
            set the output write enable

            go to _StDotProduct
        else
            inc curr_y by s
            inc out_y by 1

            go to _StResetWindow

    else
        inc curr_x by s
        inc out_x

        go to _StResetWindow
    """

    def generate(self, m: Maxpool) -> V_Block:
        max_y = m.y - m.s - m.f
        max_x = m.x - m.s - m.f

        print(max_y, max_x)

        return V_Block(
            m.out_we.set(V_Low),

            *V_If(m.curr_x > max_x)(
                m.curr_x.set(V_Low),
                m.out_x.set(V_Low),

                *V_If(m.curr_y > max_y)(
                    # m.out_we.set(V_High),
                    V_StDone

                    # _StWriteData,
                ), *V_Else(
                    m.curr_y.set(m.curr_y + m.s),
                    m.out_y.set(m.out_y + 1),

                    _StResetWindow
                )
            ), *V_Else(
                m.curr_x.set(m.curr_x + m.s),
                m.out_x.set(m.out_x + 1),

                _StResetWindow
            )
        )


class _StWriteData(V_State):
    """
    if (output addr == max)
        clear output write enable

        go to StDone
    else
        inc output addr
    """

    def generate(self, m: Maxpool) -> V_Block:
        max_out_addr = m.output_mem.size - 1

        return V_Block(

            *V_If(m.out_addr == max_out_addr)(
                m.out_we.set(V_Low),

                V_StDone
            ), *V_Else(
                m.out_addr.set(m.out_addr + 1)
            )
        )

Generated Verilog: [Layer2](https://github.com/jfw225/mnist-cnn-fpga/blob/main/deployment/DE1-SoC_Computer_15_640_current/verilog/layer2.sv)


<!-- container: dark -->
#### Layer 3: Conv2DSum1D
This layer is exactly the same as the **Conv2D** layer (section titled **Layer 1: Conv2D**). In fact, this layer runs $c=8$ iterations of **Conv2D** and takes the element-wise sum of those iterations.

The 2D convolutional layer with a 1D sum has weights $W^3$ and biases $B^3$ that are defined by
$$\begin{align*}
    & W^3:[x=2,y=2,z=8,c=8]=\begin{bmatrix} W^3_1:[x,y,z] & \dots & W^3_c:[x,y,z] \end{bmatrix}, \\ 
    & B^3:[c=8]=\begin{bmatrix} b_1 & \dots & b_c \end{bmatrix}.
\end{align*}$$

The input and output arrays are defined by 
$$\begin{align*}
    & V_3^{in}:[n=13,n=13,c=8]=V_2^{out}, \\
    & V_3^{out}:[n-1,n-1,c].
\end{align*}$$

##### Mathematical Description
Let $L^{i*}_1$ be a **Conv2D** layer with weights $W:[x,y,z]=W^3_{:,:,:,i}$ and biases $B$ s.t. $B$ is a $c$-length array of all zeros. Then we will apply $L^{i*}_1$ to each of the $0\leq i\leq c$ matrices of shape $(n,n)$ in $V_3^{in}$ and store the output in an output matrix `cout`. More rigidly, `cout` is defined by 
$$cout:[c,n-1,n-1,c]=\begin{bmatrix} L^{1*}_1\left(V_{3_{:,:,1}}^{in}\right) & \dots & L^{i*}_1\left(V_{3_{:,:,i}}^{in}\right) & \dots & L^{c*}_1\left(V_{3_{:,:,c}}^{in}\right) \end{bmatrix}$$
for $i=1,\dots,c$.

We then take the element-wise sum of each array in `cout` (note that the last dimension of `cout` is $c$) in addition to the $c$-length vector of biases in $B^3$. The output of this operation is the output of the layer and is thus stored in $V_3^{out}$:
$$V_3^{out}:[n-1,n-1,c]=cout_1+\dots+cout_i+\dots+cout_c+B^3$$
for $i=1,\dots,c$.


##### Programatic Description

In [ ]:
# python
import numpy as np

def Conv2D(
    V1in: np.ndarray[13, 13], 
    W1: np.ndarray[2, 2, 8], 
    B1: np.ndarray[8]
):  
    n, n = V1in.shape
    x, y, z = W1.shape

    # init the output array
    V1out = np.ndarray[n - 1,  n - 1, z]  # (27, 27, 8)

    for i in range(n - 1):  # rows if image
        for j in range(n - 1):  # columns of image
            # array of shape `(x, y)`
            sub: np.ndarray[x, y] = V1in[i:i + x, j:j + y] 

            # vector of length `z`
            convij: np.ndarray[z] = np.sum(sub * W1, axis=(0, 1))

            V1out[i, j] = convij + B1

    return V1out

def Conv2DSum1D(
    V3in: np.ndarray[13, 13, 8],
    W3: np.ndarray[2, 2, 8, 8],
    B3: np.ndarray[8]
):
    x, y, z, c = W3.shape
    
    cout = []
    for i in range(c):
        cout.append(Conv2D(
            V3in[:, :, i], 
            W3[:, :, :, i], 
            [0 for _ in range(c)]
        ))

    V3out = sum(cout) + B3

    return V3out

##### Verython Implementation


In [ ]:
# python
class Conv2DSum1D(Layer):
    """
    img_array of size (n, n, c)
    kernel_array of size (x, y, z, c)

    img_i = img_array[:, :, cr]
           = img_array[i + n * n * cr]
    kernel_i = kernel_array[:, :, :, cr]
             = kernel_array[i + x * y * z * cr]


    where row counter cr is swept over
    cr=0, ..., c - 1.

    img of size (n, n)
    kernel of size (x, y, z)

    We have z=8 filters, each of which are x=2 by y=2
    Let FIL_1 denote the first filter(i, j) using the notation above, then:
    kernel = [ [f1      [h1
                f2       h2
                g1 ,     j1 ...
                g2]      j2]       ]

           = [FIL_1.T, FIL_2.T, ... , FIL_8.T]  --> this is hard to think about
                                            since these filters are transposed

    at each iteration i=1,..., n - 1 and j=1,..., n - 1:
    sub(i, j) of size (x, y) is subset of the image
    sub(i, j) = [[f1, ..., fx],
                 [g1, ..., gy]]

    kernel = [[ F1 = [o1, ..., oz],
                ...
                Fx = [p1, ..., pz]],
              [ G1 = [q1, ..., qz],
                ...
                Gy = [r1, ..., rz]]

    prod(i, j) = sub(i, j) * kernel
               = [[ f1 * F1,
                    ...
                    fx * Fx],
                  [ g1 * G1,
                    gy * Gy]]
    sum(i, j) = prod.sum(axis=(0, 1)) =
        [
            f1 * o1 + ... + fx * p1 + g1 * q1 + ... + gy * r1,
            ...
            f1 * oz + ... + fx * pz + g1 * qz + ... + gy * rz
        ]

    output is array of size (n - 1, n - 1, z) where
    output[i, j] = sum(i, j)
    ------------
    This very abstract representation makes hard to develop the intuition,
    let's ignore this for now
    ------------
    mat is a 2x2 slice of the image --> img[i:i+2, j:j+2]
    Let's say:
    mat = np.array([[[0.1],
                     [0.5]],
                     [[1.],
                     [1.]]])
    kernel = np.ones( (2,2,8) )

    prod(i, j) = mat * kernel
               = np.array([[[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                            [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]],
                            [[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
                            [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ]]])
    output(i, j) = np.array([2.6, 2.6, 2.6, 2.6, 2.6, 2.6, 2.6, 2.6])
                                             --> we sum the columns

    final sum
    output of shape (o1, o2, c)
    conv_out_arr of shape (c, o1, o2, c)

    for flattened output out of len (o1 * o2 * c),
    out[oc] = sum(conv_out_arr[i * o1 * o2 * c + oc])
    for i=0, ..., c - 1,
    for oc=0, ..., o1 * o2 * c - 1.
    """

    def __init__(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray,
        input_mem: M10K,
        output_mem: M10K,
        input_shape: Optional[InputShape],
        output_shape: Optional[OutputShape]
    ):

        # create the signed mult and sigmoid modules
        self.signed_mult = SignedMult(int_width, dec_width)

        super().__init__(
            int_width=int_width,
            dec_width=dec_width,
            weights_np=weights_np,
            biases_np=biases_np,
            input_mem=input_mem,
            output_mem=output_mem,
            input_shape=input_shape,
            output_shape=output_shape,
            objects=[self.signed_mult]
        )

        # make sure input data is signed
        self.inp_data.signed = True

        # make output data a wire
        self.out_data.dtype = V_Reg

        print(self.input_shape, weights_np.shape, self.output_shape)
        n1, n2, c = self.input_shape
        assert n1 == n2
        n = n1

        x, y, z, c = weights_np.shape

        assert x == y, weights_np

        *_, o1, o2, c = self.output_shape
        assert o1 == o2

        # the constant used to scale the inputs (>> 8 -> / 256)
        self.input_scaler = 8

        # store the target size (** MIGHT BE SOURCE OF ERROR)
        self.target_size = n - 1
        print(f"{nameof(self)} Target Size: {self.target_size}")

        # the number of rows in the flattened weights
        self.w_rows = x * x

        # create registers used as iteration indices
        # ti=0, ..., target_size - 1
        self.ti = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="ti")

        # tj=0, ..., target_size - 1
        self.tj = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="tj")

        # wc=0, ..., z - 1
        self.wc = self.var(dtype=V_Reg,
                           width=self.w_addr.width,
                           name="wc")

        # cr=0, ..., c - 1
        self.cr = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="cr")

        # oc=0, ..., o1 * o2 * c - 1 (output col)
        self.oc = self.var(dtype=V_Reg,
                           width=ceil(log2(c * o1 * o2 * c)),
                           name="oc")

        # ii=0, ..., x - 1
        self.ii = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="ii")

        # jj=0, ..., x - 1
        self.jj = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="jj")

        # r=0, ..., x * x - 1
        self.r = self.var(dtype=V_Reg,
                          width=self.input_mem.addr_width,
                          name="r")

        # create a register array for the sub image
        self.sub = self.var(
            dtype=V_RegArray,
            width=self.width,
            size=x * x,
            signed=True,
            name="sub_img"
        )

        # holds the c elements of sum(i, j), used to sum
        # over the c rows of outputs
        self.sum_arr = self.var(
            dtype=V_RegArray,
            width=self.width,
            size=c,
            signed=True,
            name="sum_arr")

        # create `self.w_rows` multiplier outputs
        self.mult_outs = [self.var(dtype=V_Wire,
                                   width=self.signed_mult.width,
                                   signed=True,
                                   name=f"mult_out_{wr}")
                          for wr in range(self.w_rows)]

    @staticmethod
    def forward(
        x: np.ndarray,
        weights_np: Weights,
        biases_np: Biases
    ):

        *_, c = x.shape

        # Z1 = np.array([convolve_flat(x[:, :, i], weights_np[:, :, :, i])
        #                for i in range(c)])
        Z1 = sum([convolve_flat(x[:, :, i], weights_np[:, :, :, i])
                  for i in range(c)])

        return Z1 + biases_np

    def convert_model_params(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray
    ) -> Tuple[V_Array, V_Array]:

        assert biases_np.ndim == 1, biases_np

        w_flat = weights_np.flatten()

        weights = V_Array(
            module=self,
            dtype=V_ParameterArray,
            width=int_width + dec_width,
            size=len(w_flat),
            signed=True,
            data=[V_FixedPoint(v, int_width, dec_width) for v in w_flat],
            name=f"conv2dsum1d_weights"
        )

        biases = V_Array(
            module=self,
            dtype=V_ParameterArray,
            width=int_width + dec_width,
            size=len(biases_np),
            signed=True,
            data=[V_FixedPoint(v, int_width, dec_width)
                  for v in biases_np],
            name=f"conv2dsum1d_biases"
        )

        return weights, biases

    def generate(self) -> V_Block:
        x, y, z, c = self.weights_np.shape

        *_, o1, o2, c = self.output_shape

        vsm = V_StateMachine(_StReset, _StWaitValid, _StResetSumArr,
                             _StResetSub, _StSetInpAddr, _StSubBuffer,
                             _StSetSub, _StComputeConv, _StIncWeightIndices,
                             _StIncTargetIndices, _StWriteData, _StWriteBuffer)

        # instantiate x * x signed mult modules
        mult = self.signed_mult
        self.signed_mult_inss = [self.signed_mult.instantiate(
            self,
            (V_Empty(), mult.clk),
            (V_Empty(), mult.reset),
            (V_Empty(), mult.valid),
            (V_Empty(), mult.done),
            (V_Empty(), mult.ready),
            (self.sub[wr], mult.input_ports[0]),
            (self.weights[(self.wc * c) + self.cr +
                          (wr * z * c)], mult.input_ports[1]),
            (self.mult_outs[wr], mult.output_ports[0])
        ) for wr in range(self.w_rows)]

        return super().generate(vsm, V_Block(
            "// instantiate the multipliers",
            *[line for mult_ins in self.signed_mult_inss for line in mult_ins],

        ))


"""
Conv2DSum1D State Machine:
img of size (n, n)
kernel of size (x, y, z)


first load each input into a local register array
reasoning:
    at each iteration, we need to get 4 new inputs from input memory. this
    would take at least 5 cycles to set input addr and get new data

    getting all the inputs into a reg array would allow us to get new values
    in same cycle

call this reg array `inp_reg`
for iteration i=1,..., n - 1 and j=1,..., n - 1:
filter(i, j) is of size (x, y)


set the sub by
for ti in range(target_size):
    for tj in range(target_size):
        sub[0:x * x - 1] = inp_reg[ti:ti + x, ti:ti + x]


go straight to computing this the prod.sum:
sum(i, j) = prod.sum(axis=(0, 1)) =
        [
            f1 * o1 + ... + fx * p1 + g1 * q1 + ... + gy * r1,
            ...
            f1 * oz + ... + fx * pz + g1 * qz + ... + gy * rz
        ]
        =
        [
            sub[0] * W[0 + 0 * z] + ... + sub[wr] * W[0 + wr * z],
            sub[0] * W[1 + 0 * z] + ... + sub[wr] * W[1 + wr * z],
            ...
            sub[0] * W[wc + 0 * z] + ... + sub[wr] * W[wc + wr * z]
        ]
where wr=0, ..., x * x - 1 and is the row index and
      wc=0, ...., z - 1
"""


class _StReset(V_State):
    """
    init input read addr
    init the bias addr
    init output write addr
    lower output write enable

    init iteration variables

    go to StWaitValid
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:

        return V_Block(
            m.inp_addr.set(m.input_mem.base_addr),
            m.b_addr.set(V_Low),
            m.out_addr.set(m.output_mem.base_addr),
            m.out_we.set(V_Low),

            m.ti.set(V_Low),
            m.tj.set(V_Low),
            m.wc.set(V_Low),

            m.cr.set(V_Low),
            m.oc.set(V_Low),

            _StWaitValid
        )


class _StWaitValid(V_State):
    """
    if (valid)
        go to StInitInputReg
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:

        return V_Block(
            *V_If(m.valid)(

                _StResetSumArr
            )
        )


class _StResetSumArr(V_State):
    """
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:
        x, y, z, c = m.weights_np.shape

        return V_Block(
            *[m.sum_arr[i].set(V_Low) for i in range(c)],

            _StResetSub
        )


class _StResetSub(V_State):
    """
    reset the counters used for sub
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:
        return V_Block(
            m.ii.set(V_Low),
            m.jj.set(V_Low),
            m.r.set(V_Low),

            _StSetInpAddr
        )


class _StSetInpAddr(V_State):
    """
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:
        n = m.target_size + 1
        x, y, z, c = m.weights_np.shape

        return V_Block(
            m.inp_addr.set(
                (V_Par(m.ti + m.ii) * n * c) +
                (V_Par(m.tj + m.jj) * c) +
                m.cr
            ),

            _StSubBuffer
        )


class _StSubBuffer(V_State):
    """
    delay 1 cycle
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:
        return V_Block(

            _StSetSub
        )


class _StSetSub(V_State):
    """
    kernel of size (x, y, z, c)

    set the sub by
    for ti in range(target_size):
        for tj in range(target_size):
            sub[0:x * x - 1] = inp_reg[ti:ti + x, ti:ti + x]

            set wc to zero

            go to StComputeConv

    inp_reg of size (n, n, c)
    inp_reg[i, j, cr] = inp_reg[(i * n * c) + (j * c) + cr]
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:
        # should be the first dim of the input shape
        n = m.target_size + 1

        x, y, z, c = m.weights_np.shape

        max_ii = x - 1
        max_jj = x - 1
        max_r = x * x - 1
        indices = [(ii * n * c, jj * c) for ii in range(x) for jj in range(x)]

        return V_Block(
            # *[m.sub[r].set(m.inp_reg[
            #     (m.ti * n * c + ii) + (m.tj * c + jj) + m.cr]
            # ) for r, (ii, jj) in enumerate(indices)],
            # *[m.sub[r].set(m.inp_reg[m.tj + (jj + ii) + m.ti * n + (m.cr * n * n)])
            #   for r, (ii, jj) in enumerate(indices)],

            m.sub[m.r].set(m.inp_data),

            *V_If(m.r == max_r)(
                m.wc.set(V_Low),

                _StComputeConv
            ), *V_Else(
                m.r.set(m.r + 1),

                # dont need to handle the case of max_ii because m.r == max_r
                *V_If(m.jj == max_jj)(
                    m.ii.set(m.ii + 1),
                    m.jj.set(V_Low)
                ), *V_Else(
                    m.jj.set(m.jj + 1)
                ),

                _StSetInpAddr
            )

            # m.wc.set(V_Low),

            # _StComputeConv
        )


class _StComputeConv(V_State):
    """
    store the multiplier output in the convolutional output array
    go to StComputeSig
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:

        return V_Block(
            "// assign the output of the convolution",
            # m.conv_out_arr[m.oc].set(
            #     V_Sum(*m.mult_outs, m.biases[m.wc])),
            # m.conv_out_arr[m.oc].set(V_Sum(*m.mult_outs)),
            m.sum_arr[m.wc].set(V_Sum(m.sum_arr[m.wc], *m.mult_outs)),
            # m.out_we.set(V_High),

            _StIncWeightIndices
        )


class _StIncWeightIndices(V_State):
    """
    out_we.set(V_Low),

    if (wc == w_rows - 1)
        set wc to 0

        if (m.oc = c * o1 * o2 * c - 1)
            set m.oc to 0

            go to StIncTargetIndices
        else
            inc m.oc

            go to StIncTargetIndices
    else
        inc wc
        inc m.oc

        go to StComputeConv
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:
        *_, o1, o2, c = m.output_shape
        max_oc = c * o1 * o2 * c - 1

        # max_rows = m.w_rows - 1
        x, y, z, c = m.weights_np.shape
        max_cols = z - 1

        return V_Block(

            # m.out_we.set(V_Low),
            *V_If(m.wc == max_cols)(
                m.wc.set(V_Low),

                *V_If(m.oc == max_oc)(
                    m.oc.set(V_Low),

                    _StIncTargetIndices
                ), *V_Else(
                    # m.out_addr.set(m.out_addr + 1),
                    m.oc.set(m.oc + 1),

                    _StIncTargetIndices
                )
            ), *V_Else(
                m.wc.set(m.wc + 1),
                m.oc.set(m.oc + 1),
                # m.out_addr.set(m.out_addr + 1),

                _StComputeConv
            )
        )


class _StIncTargetIndices(V_State):
    """
    if (m.cr == c - 1)
        set m.cr to 0

        if (m.tj == m.target_size - 1)
            set m.tj to 0

            if (m.ti == m.target_size - 1)
                set m.ti to 0
                raise the output write enable

                go to _StSum1D
            else
                inc m.ti

                go to _StResetSumArr
        else
            inc m.tj

            go to _StResetSumArr

    else
        inc m.cr

        go to _StResetSub

    """

    def generate(self, m: Conv2DSum1D) -> V_Block:
        max_iter = m.target_size - 1

        x, y, z, c = m.weights_np.shape
        max_cr = c - 1

        return V_Block(
            *V_If(m.cr == max_cr)(
                m.cr.set(V_Low),


                *V_If(m.tj == max_iter)(
                    m.tj.set(V_Low),

                    *V_If(m.ti == max_iter)(
                        m.ti.set(V_Low),

                    ), *V_Else(
                        m.ti.set(m.ti + 1),

                    )
                ), *V_Else(
                    m.tj.set(m.tj + 1),

                ),

                _StWriteData
            ), *V_Else(
                m.cr.set(m.cr + 1),

                _StResetSub
            ),
        )


class _StWriteData(V_State):

    def generate(self, m: Conv2DSum1D) -> V_Block:
        max_out_addr = m.output_mem.size
        x, y, z, c = m.weights_np.shape
        max_wc = z

        return V_Block(
            # m.out_we.set(V_Low),

            m.out_data.set(m.sum_arr[m.wc] + m.biases[m.wc]),

            *V_If(m.out_addr == max_out_addr)(

                V_StDone
            ), *V_Else(
                *V_If(m.wc == max_wc)(
                    m.wc.set(V_Low),

                    _StResetSumArr
                ), *V_Else(

                    m.out_we.set(V_High),
                    _StWriteBuffer
                )
            )

        )


class _StWriteBuffer(V_State):
    """
    """

    def generate(self, m: Conv2DSum1D) -> V_Block:

        return V_Block(
            m.out_we.set(V_Low),
            # m.out_we.set(V_High),
            m.out_addr.set(m.out_addr + 1),
            m.wc.set(m.wc + 1),

            _StWriteData
        )

Generated Verilog: [Layer3](https://github.com/jfw225/mnist-cnn-fpga/blob/main/deployment/DE1-SoC_Computer_15_640_current/verilog/layer3.sv)


<!-- container: default -->
#### Layer 4: MaxpoolFC
The last layer is the same as **Maxpool** (as in the layer detailed in the section titled **Layer 2: Maxpool**) with one extra step. Unlike **Maxpool**, this layer has weights and biases defined by 
$$\begin{align*}
    W^4:[m=1352, f=10] && B^4:[f=10].
\end{align*}$$

The inputs and outputs of this layer are defined by 
$$\begin{align*}
    & V_4^{in}:[x=12,y=12,n=8]=V_3^{out}, \\
    & V_4^{out}:[f=10].
    & V_4^{out}:[\lceil x/2\rceil=6, \lceil y/2\rceil=6, n=8].
\end{align*}$$
where $\lceil r\rceil$ is $r$ rounded up to the nearest integer for some $r\in\mathbb{R}$.
We also use $s=2$ for the stride hyperparameter.


##### Mathematical Description
This layer is fairly simple. We start by applying the **Maxpool** layer with stride $s=2$ to the input and storing it in `mpout`, or rather
$$mpout:[\lceil x/2\rceil=6, \lceil y/2\rceil=6, n=8]=L_2\left(V_4^{in}\right)$$
where $\lceil r\rceil$ is $r$ rounded up to the nearest integer for some $r\in\mathbb{R}$.

We then flatten `mpout` to make it single-dimensional, take the row-wise dot product and add in the biases to get our fully-connected output:
$$V_4^{out}:[f=10]=
\begin{bmatrix} 
    B^4_1 + \sum_{j=1}^m V_{4_j}^{in}\cdot W^4_{j,1} \\
    \vdots \\
    B^4_i + \sum_{j=1}^m V_{4_j}^{in}\cdot W^4{j,i} \\ 
    \vdots \\ 
    B^4_f + \sum_{j=1}^m V_{4_j}^{in}\cdot W^4_{j,f}
\end{bmatrix}.$$


##### Programatic Description

In [ ]:
# python
import numpy as np

def Maxpool(V2in: np.ndarray[27, 27, 8], s=2):
    x, y, n = image.shape

    V2out = np.ndarray[x // 2, y // 2, n]

    # slide the window over every part of the image using stride s
    # take the maximum value at each step
    i = 0

    # slide the max pooling window vertically across the image
    while i + s <= x:
        j = 0

        # slide the max pooling window horizontally across the image
        while j + s <= y:
            # choose the max value in the window at each step
            maxij = np.max(V2in[i:i + s, j:j + s], axis=(0, 1))

            # store it to output matrix
            V2out[i // s, j // s] = maxij
            
            # increment `j`
            j += s

        # increment `i`
        i += s

    return V2out

def MaxpoolFC(
    V4in: np.ndarray[12, 12, 8],
    W4: np.ndarray[1352, 10],
    B4: np.ndarray[10],
    s=2
):
    mp = Maxpool(V4in, s)

    return mp.reshape(-1).dot(W4) + B4

##### Verython Implementation

In [ ]:
# python
class Maxpool_FC(Layer):
    """
    Let the input to this layer be 

    mp_input of size (y, x, n)
    window is of size (f * f * n)

    window = [[
        A1 = [o1, ..., on],
        ...,
        Af = [p1, ..., pn]
    ], [
        B1 = [q1, ..., qn],
        ...,
        Bf = [r1, ..., rn]
    ]]

    the output at iteration (i, j) is the columnwise max of 
    [A1, ..., Af, B1, ..., Bf], or rather, 
    max(i, j) of size (n) = [
        max(o1, ..., p1, q1, ..., r1),
        ...,
        max(on, ..., pn, qn, ..., rn)
    ] = [
        max(
            window[0 + 0 * z * z] 
            + ... + window[0 + wr * z * z] + ... +
            window[0 + (z * z - 1) * z * z]
        ), ..., 
        max(
            window[wc + 0 * z * z] 
            + ... + window[wc + wr * z * z] + ... +
            window[wc + (z * z - 1) * z * z]
        ), ..., 
        max(
            window[(n - 1) + 0 * z * z] 
            + ... + window[(n - 1) + wr * z * z] + ... +
            window[(n - 1) + (z * z - 1) * z * z]
        )
        max(window[wc + wr * z * z])
    ]

    where wc=0, ..., n - 1 is the column sweep and 
    wr=0, ..., z * z - 1 is the row sweep.
    """

    def __init__(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray,
        input_mem: M10K,
        output_mem: M10K,
        input_shape: Optional[InputShape],
        output_shape: Optional[OutputShape]
    ):
        # the number of terms used in the exponent taylor series approximation
        self.exp_num_terms = 10

        # create the max. mult, exponential, and div modules
        self.signed_max = SignedMax(int_width, dec_width)
        self.signed_mult = SignedMult(int_width, dec_width)

        super().__init__(
            int_width=int_width,
            dec_width=dec_width,
            weights_np=weights_np,
            biases_np=biases_np,
            input_mem=input_mem,
            output_mem=output_mem,
            input_shape=input_shape,
            output_shape=output_shape,
            objects=[self.signed_max, self.signed_mult]
        )

        # store f and stride values
        self._f = 2
        self._s = 2

        # we assume the f is even for now
        assert self._f % 2 == 0, self._f

        # create a register array for the sliding window
        self.window = self.var(dtype=V_RegArray,
                               width=self.width,
                               size=self.f * self.f * self.n,
                               signed=True,
                               name="window")

        # create a register for the output of the maxpool
        self.mp_out = self.var(dtype=V_RegArray,
                               width=self.width,
                               size=(self.y // 2) * (self.x // 2) * self.n,
                               signed=True,
                               name="mp_out")

        # create iteration variables as in `maxpool_flat`
        self.curr_y = self.var(dtype=V_Reg,
                               width=self.input_mem.addr_width,
                               name="curr_y")
        self.out_y = self.var(dtype=V_Reg,
                              width=ceil(log2(self.mp_out.size)),
                              name="out_y")

        self.curr_x = self.var(dtype=V_Reg,
                               width=self.input_mem.addr_width,
                               name="curr_x")
        self.out_x = self.var(dtype=V_Reg,
                              width=ceil(log2(self.mp_out.size)),
                              name="out_x")

        # jj=0, ..., f - 1
        self.jj = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="jj")

        # ii=0, ..., f - 1
        self.ii = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="ii")

        # kk=0, ..., n - 1
        self.kk = self.var(dtype=V_Reg,
                           width=self.input_mem.addr_width,
                           name="kk")

        # ww=0, ..., f * f * n - 1
        self.ww = self.var(dtype=V_Reg,
                           width=ceil(log2(self.window.size)),
                           name="ww")

        # wc=0, ..., n - 1
        self.wc = self.var(dtype=V_Reg,
                           width=self.output_mem.addr_width,
                           name="wc")

        # wr=0, ..., f * f - 1
        self.wr = self.var(dtype=V_Reg,
                           width=ceil(log2(self.window.size)),
                           name="wr")

        # holds the output of the signed max module
        self.max_out = self.var(dtype=V_Wire,
                                width=self.width,
                                signed=True,
                                name="max_out")

        # define a list to hold the output of the top level max moudle instances
        # self.max_outputs[wc] holds the max of window column wc
        self.max_outputs = list()

        # define a list to hold the max module instances
        self.max_instances = list()

        # configure the logic for the max functionality
        self._configure_max_logic()

        # create variable to hold the value of the dot product
        self.dp = self.add_var(self.signed_mult.out,
                               dtype=V_Reg, name="dot_product")

        # create variable to hold the output of the multiplier
        self.prod = self.add_var(self.signed_mult.out,
                                 dtype=V_Wire, name="prod")

    @property
    def f(self):
        """
        The `f` parameter in `maxpool_flat`. 
        TODO: KEN COME BACK AND EXPLAIN THIS
        """

        return self._f

    @property
    def s(self):
        """
        The stride parameter in `maxpool_flat`.
        """

        return self._s

    @property
    def y(self):
        """
        Input array is of size `(y, x, n)`.
        """

        y, x, n = self.input_shape

        return y

    @property
    def x(self):
        """
        Input array is of size `(y, x, n)`.
        """

        y, x, n = self.input_shape

        return x

    @property
    def n(self):
        """
        Input array is of size `(y, x, n)`.
        """

        y, x, n = self.input_shape

        return n

    @staticmethod
    def forward(
        x: np.ndarray,
        weights_np: Weights,
        biases_np: Biases
    ):

        Z2 = maxpool_flat(x)
        Z3 = Z2.reshape(-1).dot(weights_np)

        return Z3 + biases_np
        # return softmax(Z3 + biases_np)

    def convert_model_params(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        weights_np: np.ndarray,
        biases_np: np.ndarray
    ) -> Tuple[V_Array, V_Array]:

        assert biases_np.ndim == 1, biases_np

        w_flat = weights_np.T.flatten()

        weights = V_Array(
            module=self,
            dtype=V_ParameterArray,
            width=int_width + dec_width,
            size=len(w_flat),
            signed=True,
            data=[V_FixedPoint(v, int_width, dec_width) for v in w_flat],
            name=f"maxpool_weights"
        )

        biases = V_Array(
            module=self,
            dtype=V_ParameterArray,
            width=int_width + dec_width,
            size=len(biases_np),
            signed=True,
            data=[V_FixedPoint(v, int_width, dec_width)
                  for v in biases_np],
            name=f"maxpool_biases"
        )

        return weights, biases

    def generate(self) -> V_Block:
        mult = self.signed_mult

        vsm = V_StateMachine(_StReset, _StWaitValid, _StResetWindow,
                             _StSetInpAddr, _StWindowBuffer, _StSetWindow,
                             _StFindMax, _StIncMaxpool, _StWaitDotProduct, _StWriteData)

        return super().generate(vsm, V_Block(
            "// instantiate the signed maximum modules",
            *[line for sm_ins in self.max_instances for line in sm_ins],

            "\n// instantiate the multiplier",
            *mult.instantiate(
                self,
                (V_Empty(), mult.clk),
                (V_Empty(), mult.reset),
                (V_Empty(), mult.valid),
                (V_Empty(), mult.done),
                (V_Empty(), mult.ready),
                (self.mp_out[self.out_x], mult.input_ports[0]),
                (self.w_data, mult.input_ports[1]),
                (self.prod, mult.output_ports[0])
            ),
        ))

    def _configure_max_logic(self) -> None:
        """
        Configures the layer to be capable of handling the maximum 
        functionality for the window as described in _StFindMax. 
        """

        # localize `self.signed_max` for less typing
        sm = self.signed_max

        # create pairings for the maximum logic
        inds = np.arange(self.f * self.f *
                         self.n).reshape(self.f * self.f, self.n)

        def create_pairs(indices):
            # indices should always contain at least one
            n = len(indices)
            assert n > 0, indices
            if n == 1:
                return (*indices, None)
            elif n == 2:
                return (*indices,)

            return (create_pairs(indices[:2]), create_pairs(indices[2:]))

        def create_max_instances(pairs):
            """
            Recursive function that creates the required instances of 
            `SignedMax`. Each iteration returns a reference to a register in 
            `self.window` or creates a `SignedMax` instance and returns a wire 
            connected to its output. 
            """

            A, B = pairs

            if isinstance(A, int):
                # if A is an int, B can either be an int or None
                assert isinstance(B, int) or B is None, B

                # if B is None, we simply return the window register value
                if B is None:
                    return self.window[A]

                # otherwise, we set A and B to their register references
                A, B = self.window[A], self.window[B]

            # otherwise, A and B must be pairs
            else:
                assert not isinstance(B, int) and B is not None, B

                # get the corresponding output wires of pairs A and B
                A, B = create_max_instances(A), create_max_instances(B)

            # create an output wire and an instance that computes max(A, B)
            out = self.var(dtype=V_Wire,
                           width=self.width,
                           signed=True,
                           name=f"max_logic_{id_generator()}")

            self.max_instances.append(sm.instantiate(
                self,
                (V_Empty(), sm.clk),
                (V_Empty(), sm.reset),
                (V_Empty(), sm.valid),
                (V_Empty(), sm.done),
                (V_Empty(), sm.ready),
                (A, sm.input_ports[0]),
                (B, sm.input_ports[1]),
                (out, sm.output_ports[0])
            ))

            # return the output wire
            return out

        # transpose because we are taking the column-wise maximum in the window
        for col in inds.T:
            pairs = create_pairs([*map(int, col)])
            out = create_max_instances(pairs)
            self.max_outputs.append(out)


class _StReset(V_State):
    """
    init input read addr
    init weights addr to base
    init biases addr to base
    init output write addr
    lower output write enable

    init iteration variables

    init dot product and exp sum
    """

    def generate(self, m: Maxpool_FC) -> V_Block:

        return V_Block(
            m.inp_addr.set(m.input_mem.base_addr),
            m.w_addr.set(V_Low),
            m.b_addr.set(V_Low),
            m.out_addr.set(m.output_mem.base_addr),
            m.out_we.set(V_Low),

            m.curr_y.set(V_Low),
            m.out_y.set(V_Low),
            m.curr_x.set(V_Low),
            m.out_x.set(V_Low),

            m.dp.set(V_Low),

            _StWaitValid
        )


class _StWaitValid(V_State):
    """
    if (valid)
        go to StInitInputReg
    """

    def generate(self, m: Maxpool_FC) -> V_Block:

        return V_Block(
            *V_If(m.valid)(

                _StResetWindow
            )
        )


class _StResetWindow(V_State):
    """
    """

    def generate(self, m: Maxpool_FC) -> V_Block:
        return V_Block(
            m.jj.set(V_Low),
            m.ii.set(V_Low),
            m.kk.set(V_Low),
            m.ww.set(V_Low),

            _StSetInpAddr
        )


class _StSetInpAddr(V_State):
    """
    """

    def generate(self, m: Maxpool_FC) -> V_Block:

        return V_Block(
            m.inp_addr.set(
                (V_Par(m.curr_y + m.jj) * m.x * m.n) +
                (V_Par(m.curr_x + m.ii) * m.n) +
                m.kk),

            _StWindowBuffer
        )


class _StWindowBuffer(V_State):
    """
    """

    def generate(self, m: Maxpool_FC) -> V_Block:

        return V_Block(

            _StSetWindow
        )


class _StSetWindow(V_State):
    """
    set window = inp_reg[curr_y:curr_y + f, curr_x:curr_x + f]

    inp_reg of size(y, x, n)
    inp_reg[j, i, k] = inp_reg[(j * x * n) + (i * n) + k]

    need to sweep
    j=curr_y, ..., curr_y + f
    i=curr_x, ..., curr_x + f
    k=0, ..., n

    go to _StFindMax
    """

    def generate(self, m: Maxpool_FC) -> V_Block:
        max_jj = m.f - 1
        max_ii = m.f - 1
        max_kk = m.n - 1
        max_ww = m.f * m.f * m.n - 1

        return V_Block(
            m.window[m.ww].set(m.inp_data),

            *V_If(m.ww == max_ww)(

                _StFindMax
            ), *V_Else(
                m.ww.set(m.ww + 1),

                *V_If(m.kk == max_kk)(
                    m.kk.set(V_Low),

                    # dont need to set jj low because that is case
                    # where m.ww == max_ww
                    *V_If(m.ii == max_ii)(
                        m.ii.set(V_Low),
                        m.jj.set(m.jj + 1)
                    ), *V_Else(
                        m.ii.set(m.ii + 1)
                    )
                ), *V_Else(
                    m.kk.set(m.kk + 1)
                ),

                _StSetInpAddr
            )
        )


class _StFindMax(V_State):
    """
    assign each of the column-wise maximums to the mp_out register

    mp_out of size (y // 2, x // 2, n)

    mp_out[(out_y * x // 2 * n) + (out_x * n) + wc] = max_outputs[wc]
    for window column index wc=0,..., n - 1.
    """

    def generate(self, m: Maxpool_FC) -> V_Block:

        return V_Block(
            *[m.mp_out[
                (m.out_y * (m.x // 2) * m.n) + (m.out_x * m.n) + wc
            ].set(m.max_outputs[wc])
                for wc in range(m.n)],

            _StIncMaxpool
        )


class _StIncMaxpool(V_State):
    """
    set each of the previously set mp_outs to to themselves

    if (curr_x <= x - s - f)
        set curr_x to 0
        set out_x to 0

        if (curr_y <= y - s - f)
            go to _StDotProduct
        else
            inc curr_y by s
            inc out_y by 1

            go to _StSetWindow

    else
        inc curr_x by s
        inc out_x

        go to _StSetWindow
    """

    def generate(self, m: Maxpool_FC) -> V_Block:
        max_y = m.y - m.s - m.f
        max_x = m.x - m.s - m.f

        print(max_y, max_x)

        inds = [(m.out_y * (m.x // 2) * m.n) +
                (m.out_x * m.n) + wc for wc in range(m.n)]

        return V_Block(
            *[m.mp_out[ind].set(m.mp_out[ind]) for ind in inds],

            *V_If(m.curr_x > max_x)(
                m.curr_x.set(V_Low),
                m.out_x.set(V_Low),

                *V_If(m.curr_y > max_y)(

                    _StWaitDotProduct
                ), *V_Else(
                    m.curr_y.set(m.curr_y + m.s),
                    m.out_y.set(m.out_y + 1),

                    _StResetWindow
                )
            ), *V_Else(
                m.curr_x.set(m.curr_x + m.s),
                m.out_x.set(m.out_x + 1),

                _StResetWindow
            )
        )


class _StWaitDotProduct(V_State):
    """
    ** use out_x to iterate over the mp_out array

    - add prod to dot product

    - if (out_x == max - 1)
        - inc weights addr

        - raise the output write enable
        - set the output data to dp + prod + bias

        - go to StWaitExponential
    - else
        - inc out x
        - inc weights addr
    """

    def generate(self, m: Maxpool_FC) -> V_Block:
        max_mp_out_addr = m.mp_out.size - 1

        return V_Block(
            m.dp.set(m.dp + m.prod),
            "\n",
            *V_If(m.out_x == max_mp_out_addr)(
                m.w_addr.set(m.w_addr + 1),
                m.out_we.set(V_High),
                m.out_data.set(m.dp + m.prod + m.biases[m.out_addr]),

                _StWriteData
            ), *V_Else(
                m.out_x.set(m.out_x + 1),
                m.w_addr.set(m.w_addr + 1),
            )
        )


class _StWriteData(V_State):
    """
    lower output write enable
    clear out_x
    clear dot product

    if (output addr == max)
        go to StDone
    else
        inc out addr

        go to _StWaitDotProduct
    """

    def generate(self, m: Maxpool_FC) -> V_Block:
        max_out_addr = m.output_mem.size - 1

        return V_Block(
            m.out_we.set(V_Low),
            m.out_x.set(V_Low),
            m.dp.set(V_Low),

            *V_If(m.out_addr == max_out_addr)(

                V_StDone
            ), *V_Else(
                m.out_addr.set(m.out_addr + 1),

                _StWaitDotProduct
            )
        )

Generated Verilog: [Layer4](https://github.com/jfw225/mnist-cnn-fpga/blob/main/deployment/DE1-SoC_Computer_15_640_current/verilog/layer4.sv)


<!-- container: dark -->
#### The Model
Now that each layer is built, we need a module to instantiate the layers and expose to the top-level module (i.e. this is the module that gets instantiated in the DE1 SoC file).

##### Verython Implementation


In [ ]:
# python
class Model(V_Target):
    def __init__(
        self,
        int_width: BitWidth,
        dec_width: BitWidth,
        input_mem: M10K,
        output_mem: M10K,
        *specs: Iterable[LayerSpec]
    ):

        self.int_width = int_width
        self.dec_width = dec_width
        self.width = int_width + dec_width

        assert self.width == input_mem.width and self.width == output_mem.width
        assert len(specs) > 0

        self.input_mem = input_mem
        self.output_mem = output_mem

        super().__init__()

        self._configure_ports()

        # ordered list of layers
        self.layers: List[Layer] = []

        # {Layer: (layer_cs, input_mem, output_mem)}
        self.layer_map: Dict[Layer, Tuple[V_ConnSpec[Layer], M10K, M10K]] = {}

        self.memories: Dict[M10K, V_ConnSpec[M10K]] = {
            self.input_mem: V_ConnSpec[M10K](self,
                                             self.input_mem,
                                             prefix="input_mem",
                                             clk=self.clk,
                                             reset=self.reset,
                                             write_en=V_Empty(),
                                             read_addr=self.inp_addr,
                                             write_addr=V_Empty(),
                                             read_data=self.inp_data,
                                             write_data=V_Empty()
                                             ),
            self.output_mem: V_ConnSpec[M10K](self,
                                              self.output_mem,
                                              prefix="output_mem",
                                              clk=self.clk,
                                              reset=self.reset,
                                              write_en=self.out_we,
                                              read_addr=V_Empty(),
                                              write_addr=self.out_addr,
                                              read_data=V_Empty(),
                                              write_data=self.out_data
                                              )
        }

        self._set_up_layers(*specs)

        # set each layer reset flag to be a reg
        for layer_cs, *_ in self.layer_map.values():
            layer_cs.reset.dtype = V_Reg

        # create each of the layer states
        self.layer_states: List[Type[V_State]] = self._create_layer_states(0)

    def generate(self) -> V_Block:
        transition_memories = {mem: cs for mem, cs in self.memories.items()
                               if mem not in [self.input_mem, self.output_mem]}

        vsm = V_StateMachine(_StReset, _StWaitValid, *self.layer_states)

        self.layer_ins = [layer.instantiate(
            self, *cs) for layer, (cs, *_) in self.layer_map.items()]

        return V_Block(
            "\n// the model state machine",
            *vsm.generate(self, self.clk, self.reset, self.done),

            "\n// instantiate the transition memories",
            *[line for mem, cs in transition_memories.items()
              for line in mem(self, *cs)],

            "\n// instantiate the layers",
            *[line for layer_ins in self.layer_ins for line in layer_ins]
        )

    def instantiate(
        self,
        instantiator: V_Module,
        clk: V_Clock,
        reset: V_Reset,
        valid: V_Valid,
        done: V_Done,
        ready: V_Ready
    ):
        assert isinstance(instantiator, V_Module)
        assert V_Clock.isinstance(clk)
        assert V_Reset.isinstance(reset)
        assert V_Valid.isinstance(valid)
        assert V_Done.isinstance(done)
        assert V_Ready.isinstance(ready)

        port_connections = [
            (clk, self.clk),
            (reset, self.reset),
            (valid, self.valid),
            (done, self.done),
            (ready, self.ready)
        ]

        inp_addr, inp_data = self.input_mem.read

        assert instantiator.name in self.input_mem.connections, (
            f"{instantiator.name} has not instantiated {self.input_mem} yet.")

        connections = self.input_mem.connections[instantiator.name]
        [addr_conn] = filter(lambda conn: conn.port is inp_addr,
                             connections)
        [data_conn] = filter(lambda conn: conn.port is inp_data,
                             connections)

        # save port connections
        port_connections += [(addr_conn.var, self.inp_addr),
                             (data_conn.var, self.inp_data)]

        write_addr, write_data, write_en = self.output_mem.write

        assert instantiator.name in self.output_mem.connections, (
            f"{instantiator.name} has not instantiated {self.output_mem} yet.")

        connections = self.output_mem.connections[instantiator.name]
        [addr_conn] = filter(lambda conn: conn.port is write_addr,
                             connections)
        [data_conn] = filter(lambda conn: conn.port is write_data,
                             connections)
        [we_conn] = filter(lambda conn: conn.port is write_en,
                           connections)

        # save port connections
        port_connections += [(addr_conn.var, self.out_addr),
                             (data_conn.var, self.out_data),
                             (we_conn.var, self.out_we)]

        return super().instantiate(instantiator, *port_connections)

    def _configure_ports(self):
        """
        Creates all of the necessary ports.
        """

        # configure the input ports
        inp_addr, inp_data = self.input_mem.read

        # create local copies
        self.inp_addr = self.add_port(inp_addr,
                                      port_type=V_Output,
                                      dtype=V_Reg,
                                      name=f"inp_addr")

        self.inp_data = self.add_port(inp_data,
                                      port_type=V_Input,
                                      dtype=V_DType,
                                      name=f"inp_data")

        # configure the output ports
        out_addr, out_data, out_we = self.output_mem.write

        # create local copies
        self.out_addr = self.add_port(out_addr,
                                      port_type=V_Output,
                                      dtype=V_Reg,
                                      name=f"out_addr")

        self.out_data = self.add_port(out_data,
                                      port_type=V_Output,
                                      dtype=V_Reg,
                                      name=f"out_data")

        self.out_we = self.add_port(out_we,
                                    port_type=V_Output,
                                    dtype=V_Reg,
                                    name=f"out_we")

    def _set_up_layers(self, *specs: Iterable[LayerSpec]) -> None:
        """
        Sets up each layer with the appropriate input/output objects.
        """

        # set initial I/O objects
        input_mem, output_mem = self.input_mem, None

        # create a list to hold the objects that need to get created
        objects = list()

        for i, (LayerT, weights_np, biases_np,
                input_shape, output_shape) in enumerate(specs):
            print(f"Creating Layer: {LayerT}")

            input_size = np.prod(input_shape)
            output_size = np.prod(output_shape)

            assert_msg = f"{input_mem} must have size of {input_size}"
            assert input_mem.size == input_size, assert_msg

            # get the input mem conn spec
            im_cs = self.memories[input_mem]

            # skip last layer
            if i + 1 == len(specs):
                break

            # create new output memory
            output_mem = M10K(self.width, output_size, name=f"trans_mem{i}")

            # add output memory to `objects` so it's generated
            objects.append(output_mem)

            # create layer object
            layer: Layer = LayerT(int_width=self.int_width,
                                  dec_width=self.dec_width,
                                  weights_np=weights_np,
                                  biases_np=biases_np,
                                  input_mem=input_mem,
                                  output_mem=output_mem,
                                  input_shape=input_shape,
                                  output_shape=output_shape)

            # create the output mem conn spec
            om_cs = V_ConnSpec[M10K](
                self,
                output_mem,
                prefix=output_mem.name,
                clk=self.clk,
                reset=self.reset
            )

            layer_cs = V_ConnSpec[Layer](self,
                                         layer,
                                         prefix=layer.name,
                                         clk=self.clk,
                                         inp_addr=im_cs.read_addr,
                                         inp_data=im_cs.read_data,
                                         out_addr=om_cs.write_addr,
                                         out_data=om_cs.write_data,
                                         out_we=om_cs.write_en)

            # adjust layer cs local variable dtypes
            layer_cs.valid.dtype = V_Reg

            # get layer file writer
            layer_file = layer.tofile(f"layer{i + 1}")

            # include layer
            self.include(layer_file)

            # store the output mem conn spec
            self.memories[output_mem] = om_cs

            # add layer instance to list
            self.layers.append(layer)

            # store an association between the layer and it's I/O memories
            self.layer_map[layer] = (layer_cs, input_mem, output_mem)

            # update input memory
            input_mem = output_mem

        ###############

        # set the objects field of `self`
        self._objects = objects

        # get the conn spec of the output memory
        om_cs = self.memories[self.output_mem]

        # create the last layer
        layer: Layer = LayerT(int_width=self.int_width,
                              dec_width=self.dec_width,
                              weights_np=weights_np,
                              biases_np=biases_np,
                              input_mem=input_mem,
                              output_mem=self.output_mem,
                              input_shape=input_shape,
                              output_shape=output_shape)

        # create the last conn spec
        layer_cs = V_ConnSpec[Layer](self,
                                     layer,
                                     prefix=layer.name,
                                     clk=self.clk,
                                     inp_addr=im_cs.read_addr,
                                     inp_data=im_cs.read_data,
                                     out_addr=om_cs.write_addr,
                                     out_data=om_cs.write_data,
                                     out_we=om_cs.write_en)

        # adjust layer cs local variable dtypes
        layer_cs.valid.dtype = V_Reg

        # add last layer instance to list
        self.layers.append(layer)

        # store last association between the layer and it's I/O memories
        self.layer_map[layer] = (layer_cs, input_mem, output_mem)

        # get layer file writer of last layer
        layer_file = layer.tofile(f"layer{i + 1}")

        # include last layer
        self.include(layer_file)

    def _create_layer_states(self, i) -> List[Type[V_State]]:

        def generate(self, m: Model) -> V_Block:
            """
            clear layer valid
            if (layer done)
                clear next layer reset
                set next layer valid

                go to next layer state or st done
            """

            layer = m.layers[i]
            layer_cs, *_ = m.layer_map[layer]

            if i == len(m.layers) - 1:
                block = V_Block(V_StDone)
            else:
                next_layer = m.layers[i + 1]
                next_layer_cs, *_ = m.layer_map[next_layer]

                block = V_Block(
                    next_layer_cs.reset.set(V_Low),
                    next_layer_cs.valid.set(V_High),
                    m.layer_states[i + 1]
                )

            return V_Block(
                layer_cs.valid.set(V_Low),
                *V_If(layer_cs.done)(
                    *block
                )
            )

        st = type(f"_StWaitLayer{i + 1}Done",
                  (V_State, ), {"generate": generate})

        if i == len(self.layers) - 1:
            return [st]

        return [st] + self._create_layer_states(i + 1)


"""
Model State Machine:
"""


class _StReset(V_State):
    """
    set each layer reset
    clear each layer valid

    go to StWaitValid
    """

    def generate(self, m: Model) -> V_Block:

        return V_Block(
            *[cs.reset.set(V_High) for cs, *_ in m.layer_map.values()],
            *[cs.valid.set(V_Low) for cs, *_ in m.layer_map.values()],

            _StWaitValid
        )


class _StWaitValid(V_State):
    """
    if (valid)
        clear layer1 reset
        set layer1 valid

        go StWaitLayer1Done
    """

    def generate(self, m: Model) -> V_Block:
        assert len(m.layer_states) > 0

        layer1 = m.layers[0]
        layer_cs, *_ = m.layer_map[layer1]

        return V_Block(
            *V_If(m.valid)(
                layer_cs.reset.set(V_Low),
                layer_cs.valid.set(V_High),

                m.layer_states[0]
            )
        )


Generated Verilog: [Model](https://github.com/jfw225/mnist-cnn-fpga/blob/main/deployment/DE1-SoC_Computer_15_640_current/verilog/model.sv)


<!-- container: default -->
## Results and Conclusions
Ultimately, we are really happy with the outcome of the project. There were essentially two parts of the project: training a good model and putting our model on the FPGA. The latter we were able to test emperically using ***Verython***. We built a feature that calculated the mean squared error between the expected output and the output of our Verilog code. We found that our verilog output differs by about $10^{-3}$ from the output of our Python model. And to be clear, this gives no indication as to how well our model can classify images of digits. That number simply says that our Verilog implementation was about $\frac{1}{1000}$ off from the value that we would have computed if the same operations were performed on desktop computer. With that being said, the precision that we observed is more than enough for our FPGA implementation to match the digit classification of the Python implementation 99.99% of the time.

That's all to say that if you drew a six, the Python model predicted a zero, and the FPGA predicted a zero, we would be ecstatic. However, Kenneth Trinh (klt45) managed to train an extremely accurate model as well. So not only is there very little precision lost on the FPGA, the FPGA is able to accurately classify drawings of digits. So yeah, I'd say we are pretty happy with the result. Here is a short clip of Dr. Adams testing out the system:
![video][hunter drawing inputs](https://www.youtube.com/watch?v=ZbLdQtHRYA0)

The entire project was inspired, designed, and developed, by our team. And even though the specific model that we trained might not be widely applicable, ***Verython*** can be used to tackle a wide variety of problems. That is why shortly after we turn in this project, we are going to add ***Verython*** to the PyPi repository and make it open-source. Our hope is that we can make the library robust enough to make high performance computing more accessible to everyone.


## Appendix

### Appendix A

The group approves this report for inclusion on the course website.

The group approves the video for inclusion on the course YouTube channel.


### Appendix B: Verython
***[Verython](https://github.com/jfw225/verython)*** (portmanteau of Verilog and Python pronounced "verithon").

##### Verython Types
[V_Types](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vtypes.py)

##### Verython Syntax
[V_Syntax](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vsyntax.py)

##### Verython Objects
[V_Objects](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vsyntax.py)

##### Verython Modules
[V_Module](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vmodule.py)
[V_Iterable](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/viterable.py)
[V_Target](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vtarget.py)

##### Verython States and State Machines
[V_State and V_StateMachine](https://github.com/jfw225/mnist-cnn-fpga/blob/main/src/python/verilog/core/vstate.py)


### Appendix C
[Comparison of deep learning and human observer performance for detection and characterization of simulated lesions](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6586983/)